In [1]:
!pip install geemap

In [2]:
!pip install geedim

In [3]:
import ee
import geemap
import os
import geedim
import pandas as pd 

In [4]:
ee.Authenticate()
ee.Initialize(project='ee-pradnyauoa')

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


In [5]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [6]:
cty = ee.FeatureCollection('projects/ee-pradnyauoa/assets/data')

In [7]:
roi = cty.geometry()
print(roi)

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "Collection.geometry",
    "arguments": {
      "collection": {
        "functionInvocationValue": {
          "functionName": "Collection.loadTable",
          "arguments": {
            "tableId": {
              "constantValue": "projects/ee-pradnyauoa/assets/data"
            }
          }
        }
      }
    }
  }
})


In [8]:
worldGeometry = ee.Geometry.Polygon([
  [-180, 90],  #// Top-left corner
  [180, 90],   #// Top-right corner
  [180, -90],  #// Bottom-right corner
  [-180, -90]  #// Bottom-left corner
])

In [9]:
import numpy as np
import ee

#g3 = ee.Geometry.Polygon([
#    [-180, 90],  #// Top-left corner
#    [0, 90],   #// Top-right corner
#    [0, -90],  #// Bottom-right corner
#    [-180, -90]  #// Bottom-left corner
#])

no_of_tiles=6
scl=300
scl_text = "30" #"300"

fl =  ( ["UHI_DaytimeNighttime_300_"+scl_text, "UHI_Daytime_300_"+scl_text, "UHI_Nighttime_300_"+scl_text, "UHIpxAnnual_2018_DayNightBandMean_300m_scale"+scl_text, "SurfaceWater_2018_30m_"+scl_text] )

citylist= list()
gstart = np.ndarray(6) #no_of_tiles)
gend = np.ndarray(6) #no_of_tiles)

g = list()
for i in range(6):
  sect = (360/no_of_tiles)
  start = -180+ (sect* (i))
  end = -180+ (sect* (i+1))


  gstart[i] = start
  gend[i] = end
  print(start, end)
  #print(gstart[i], gend[i])
  gm = ( ee.Geometry.Polygon([
      [ gstart[i], 85],  #// Top-left corner
      [ gend[i], 85],   #// Top-right corner
      [ gend[i], -85],  #// Bottom-right corner
      [ gstart[i], -85]  #// Bottom-left corner
      ]) )
  #print(gm)
  g.append( gm)
#print(g[0])

-180.0 -120.0
-120.0 -60.0
-60.0 0.0
0.0 60.0
60.0 120.0
120.0 180.0


In [10]:
print(g[0])

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "GeometryConstructors.Polygon",
    "arguments": {
      "coordinates": {
        "constantValue": [
          [
            [
              -180.0,
              85
            ],
            [
              -120.0,
              85
            ],
            [
              -120.0,
              -85
            ],
            [
              -180.0,
              -85
            ]
          ]
        ]
      },
      "evenOdd": {
        "constantValue": true
      }
    }
  }
})


In [21]:
citylist= list()
cty = ee.FeatureCollection('projects/ee-pradnyauoa/assets/data')
# Filter cities that intersect with the image
#intersecting_cities = cty.filterBounds(image2.geometry())

for k in range(6):  # no_of_tiles):
    if (k < 4):
        ctylst = list()
        ctylst= pd.read_csv('l'+str(k)+'.csv', encoding="utf-8").values.tolist()
        citylist.append(ctylst )
        continue
    #k=0
    print('section=', k)
    filterInside = ee.Filter.bounds(g[k]);
    tbl = cty.filter(filterInside)
    ftrs = tbl.toList(tbl.size())
    fld = 'name_conve'
    # Print the specific property of each feature
    ctylst=list()
    for i in range(0, ftrs.size().getInfo()):
        print(i)
        feature = ee.Feature(ftrs.get(i))
        ctynm = feature.get(fld).getInfo()
        ctylst.append(ctynm)
    print(len(ctylst))
    citylist.append(ctylst )
    print(citylist[k])
    df= pd.DataFrame( ctylst)
    df.to_csv('l'+str(k)+'.csv', index=False, encoding="utf-8")


section= 4
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274

In [ ]:
print(len(ctylst))

In [37]:
print(citylist[1])


[['Brochet'], ['Lutselke'], ['Salluit'], ['Schefferville'], ['Stony Rapids'], ['Calais1'], ['Chonchi'], ['Comondante Luis Piedrabuena'], ['Bocas del Toro'], ['Cuevo'], ['Guaranda'], ['Jaque'], ['Muisne'], ['San Juan del Sur'], ['Tarapoto'], ['Tarma'], ['Tournavista'], ['Paso Rio Mayo'], ['Pukatawagan'], ['Wallace'], ['Eastmain'], ['Hopedale'], ['Dangriga'], ['Nacaome'], ['Quime'], ['Sica Sica'], ['Tinogasta'], ['Trancas'], ['Calbuco'], ['Chosica'], ['Otuzco'], ['Thessalon'], ['Betanzos'], ['Cap-Chat'], ['Comallo'], ['Combarbala'], ['Esperanza'], ['Loreto'], ['Tarabuco'], ['Villa Rumipal'], ['Baddeck'], ['Tonopah'], ['Chumbicha'], ['Huaura'], ['Maria Elena'], ['San Luis1'], ['Uncia'], ['Van Horn'], ['Vicente Guerrero'], ['Santa Barbara3'], ['Villarica'], ['White Sulphur Springs'], ['Geraldton2'], ['Salinas2'], ['Urubamba'], ['Uspallata'], ['General Conesa'], ['Lac La Biche'], ['Almirante'], ['Lamas'], ['Paraiso'], ['Peto'], ['Puquio'], ['Santo Tomas'], ['Gimli'], ['Lead'], ['Balancan'],

In [39]:
l1= citylist[0]
l2= citylist[1]
l3= citylist[2]
l4= citylist[3]
l5= citylist[4]
l6= citylist[5]

In [ ]:
#l1 = list(['Eagle', 'Emmonak', 'Lavrentiya', 'Mekoryuk', 'Nenana', 'Teller', 'Sand Point', 'Mendocino', 'Selawik', 'Tununak', 'Dillingham', 'Palmer', 'Tillamook', 'Terrace', 'Courtenay', 'Arcata', 'Centralia', 'Paso Robles', 'Springfield5', 'Medford', 'Vallejo', 'Monterey1', 'Wahiawa', 'Monterey2', 'Berkeley', 'San Mateo', 'Kaktovik', 'Chignik', 'Hooper Bay', 'Karluk', 'McGrath', 'Togiak', 'Wales', 'Yakutat', 'Quinhagak', 'Shishmaref', 'Wainwright', 'Fort Simpson', 'Hoonah', 'King Salmon', 'Koyuk', 'Unalakleet', 'Unalaska', 'Kotlit', 'Norman Wells', 'Skagway', 'Provideniya', 'Wrangell', 'Cordova', 'Port Hardy', 'Fort Yukon', 'Kotzebue', 'Burns Lake', 'Neiafu', 'Mt. Shasta', 'Lillooet', 'Homer', 'Powell River', 'Smithers', 'Rarotonga', 'Kenai', 'Ketchikan', 'Astoria', 'Crescent City', 'Dawson Creek', 'Lihue', 'Williams Lake', 'Pago Pago', 'Campbell River', 'Ukiah', 'Wailuku', 'Aberdeen2', 'Coos Bay', 'Roseburg', 'San Luis Obispo', 'Corvallis', 'Papeete', 'Klamath Falls', 'Grants Pass', 'Albany3', 'Kamloops', 'Wenatchee', 'Salinas1', 'Longview2', 'Merced', 'Chico', 'Yakima', 'Nanaimo', 'Santa Maria1', 'Bend', 'Bellingham', 'Prince George', 'Yuba City', 'Redding', 'Bremerton', 'Eugene', 'Santa Rosa1', 'Modesto', 'Oakland', 'Everett', 'Tacoma', 'Point Hope', 'Sitka', 'Fort St. John', 'Eureka', 'Hilo', 'Salem2', 'Olympia', 'Honolulu', 'Vancouver1', 'San Jose3', 'Fort Good Hope', 'Valdez', 'Nome', 'Barrow', 'Bethel', 'Kodiak', 'Nukualofa', 'Whitehorse', 'Juneau', 'Apia', 'Fairbanks', 'Victoria2', 'Anchorage', 'Vancouver2', 'Portland2', 'Sacramento', 'Seattle', 'San Francisco1'])
#l2= list(['Brochet', 'Lutselke', 'Salluit', 'Schefferville', 'Stony Rapids', 'Calais1', 'Chonchi', 'Comondante Luis Piedrabuena', 'Bocas del Toro', 'Cuevo', 'Guaranda', 'Jaque', 'Muisne', 'San Juan del Sur', 'Tarapoto', 'Tarma', 'Tournavista', 'Paso Rio Mayo', 'Pukatawagan', 'Wallace', 'Eastmain', 'Hopedale', 'Dangriga', 'Nacaome', 'Quime', 'Sica Sica', 'Tinogasta', 'Trancas', 'Calbuco', 'Chosica', 'Otuzco', 'Thessalon', 'Betanzos', 'Cap-Chat', 'Comallo', 'Combarbala', 'Esperanza', 'Loreto', 'Tarabuco', 'Villa Rumipal', 'Baddeck', 'Tonopah', 'Chumbicha', 'Huaura', 'Maria Elena', 'San Luis1', 'Uncia', 'Van Horn', 'Vicente Guerrero', 'Santa Barbara3', 'Villarica', 'White Sulphur Springs', 'Geraldton2', 'Salinas2', 'Urubamba', 'Uspallata', 'General Conesa', 'Lac La Biche', 'Almirante', 'Lamas', 'Paraiso', 'Peto', 'Puquio', 'Santo Tomas', 'Gimli', 'Lead', 'Balancan', 'Happy Valley', 'Hardin', 'General Roca', 'Liverpool1', 'Sonson', 'Villanueva', 'Collipulli', 'Abra Pampa', 'Canatlan', 'Pampa del Infierno', 'Puerto Baquerizo Moreno', 'San Carlos', 'Nueva Imperial', 'Quirihue', 'Boaco', 'Motupe', 'El Maiten', 'Loncoche', 'Dillon', 'Acatlan', 'Camiri', 'Polson', 'Portachuelo', 'San Antonio de los Banos', 'Uyuni', 'Vegreville', 'Chepes', 'Villa Union', 'Ely1', 'Chuquicamata', 'Flin Flon', 'Powell', 'Viacha', 'Ascension', 'Goodland', 'Melville', 'Nephi', 'Pedro Luro', 'San Jose del Guaviare', 'Tekax', 'Atoyac', 'Rio Bueno', 'Houlton', 'San Gabriel', 'Stettler', 'La Ligua', 'Nautla', 'Chilecito', 'Dalhart', 'Mollendo', 'Selkirk', 'Childress', 'La Sarre', 'Weyburn', 'New Liskeard', 'Puerto Varas', 'La Ronge', 'Guines', 'Richmond2', 'Canas', 'Montpelier2', 'San Antonio Oeste', 'Victoria5', 'Curanilahue', 'Lander', 'Caibarien', 'Miahuatlan', 'Amherst', 'Miles City', 'Allende', 'Alpine', 'General Guemes', 'Camrose', 'Devils Lake', 'Dryden', 'Ironwood', 'Puerto Aisen', 'Amos', 'Punata', 'Wetaskiwin', 'Santa Rosa de Copan', 'Alliance', 'Santiago Ixcuintla', 'Kapuskasing', 'Truth or Consequences', 'La Union', 'Pichilemu', 'Baie-Comeau', 'Neiba', 'Progreso', 'Glendive', 'Bell Ville', 'San Felipe2', 'Ingeniero Guillermo N. Juarez', 'Crookston', 'Sabinas Hidalgo', 'Esquel', 'Glenwood Springs', 'Cerro de Pasco', 'Chacabuco', 'Coronel Suarez', 'Lincoln2', 'Cauquenes', 'Villa Constitucion', 'Arjona', 'Ontario', 'Castro1', 'Ferrenafe', 'Sensuntepeque', 'Spencer', 'Autlan', 'Motul', 'Puerto Barrios', 'San Ramon de la Nueva Oran', 'Bartlesville', 'Nueva Rosita', 'Veinticinco de Mayo', 'Hinton', 'Penonome', 'Tomah', 'La Grande', 'Vernon', 'Campoalegre', 'Zapala', 'Safford', 'Angol', 'Dolbeau', 'Alta Gracia', 'Esteli', 'Tres Arroyos', 'Ciudad Constitucion', 'Huacho', 'Tizimin', 'Coban', 'La Ceiba', 'Nueve de Julio', 'Yacuiba', 'Cliza', 'Tartagal', 'Las Vegas1', 'Deming', 'Montemorelos', 'Palma Soriano', 'Hereford', 'Kirksville', 'San Fernando3', 'Willmar', 'North Battleford', 'Burley', 'Valle de la Pascua', 'San Francisco2', 'Guasdualito', 'Hays', 'Villa Angela', 'Douglas4', 'Sogamoso', 'Riviere-du-Loup', 'Bowling Green2', 'Cranbrook', 'Juigalpa', 'Iron Mountain', 'Manzanillo1', 'Apatzingan', 'Linares1', 'Albert Lea', 'Plattsburg', 'Chivilcoy', 'Puerto Madryn', 'Chitre', 'Ft. Dodge', 'Alice', 'Delano', 'Ciudad Mante', 'Punta Alta', 'Garden City', 'Rouyn-Noranda', 'Alpena', 'Huamachuco', 'Ayacucho', 'Faribault', 'Brockville', 'Ciego de Avila', 'Kearney', 'New Glasgow', 'McAlester', 'Rio Verde1', 'Ardmore', 'Iguala', 'Palatka', 'Brownwood', 'Ciudad Hidalgo', 'Waterville', 'Carora', 'Chilpancingo', 'Trelew', 'Carupano', 'Atlixco', 'Cienfuegos', 'Ocumare del Tuy', 'Cutral Co', 'Ottumwa', 'San Francisco de Macoris', 'Clarksburg', 'Magangue', 'Brainerd', 'Galesburg', 'Juchitan', 'Stillwater', 'Paragould', 'Big Spring', 'Jutiapa', 'Villa Maria', 'Villavicencio', 'Cartago2', 'Prince Albert', 'Rafaela', 'Barrancabermeja', 'Hidalgo del Parral', 'Casa Grande', 'Ciudad Madero', 'Presidencia Roque Saenz Pena', 'Cleburne', 'Coronel', 'Quillota', 'Laurel', 'Madisonville', 'Harrisonburg', 'Valera', 'Marion', 'Waycross', 'Olavarria', 'Tulare', 'Ciudad Valles', 'Orangeburg', 'Villa Carlos Paz', 'San Martin1', 'Lumberton', 'Los Teques', 'Lake City', 'Juliaca', 'Oruro', 'Sabanalarga', 'Portoviejo', 'Crestview', 'Manhattan', 'Orillia', 'Bello', 'Sheboygan', 'Guanare', 'La Grange', 'Tuquerres', 'Anaco', 'San Marcos', 'New Iberia', 'Mankato', 'Joliette', 'Cape Girardeau', 'Temuco', 'Rio Cuarto', 'New Braunfels', 'Cartago1', 'Hopkinsville', 'Farmington', 'Jaltipan', 'Zanesville', 'Lancaster2', 'Richmond3', 'Drummondville', 'Callao', 'San Juan del Rio', 'Belleville2', 'Enid', 'Holguin', 'San Luis2', 'Williamsport', 'Hutchinson', 'Meridian', 'Grand Island', 'Pittsfield', 'Florence2', 'Salisbury', 'San Nicolas', 'San Miguel', 'Girardot', 'Valledupar', 'San Salvador de Jujuy', 'Talcahuano', 'Victoria1', 'St. Charles2', 'Leesburg', 'Janesville', 'Azogues', 'Blacksburg', 'Lawrence', 'Annapolis', 'San Cristobal de Las Casas', 'Parkersburg', 'Xalapa', 'Peterborough1', 'Saratoga Springs', 'Parana', 'Minatitlan', 'Iowa City', 'Palm Coast', 'St.-Jerome', 'La Crosse', 'Lake Havasu City', 'Altoona', 'Barinas', 'Freeport1', 'Rocky Mount', 'Homestead', 'Las Heras', 'Texas City', 'Springfield3', 'Celaya', 'Eau Claire', 'La Vega', 'Greeley', 'Kingsport', 'Trois Rivieres', 'Sonsonate', 'Gomez Palacio', 'Benton Harbor', 'Saginaw2', 'Dalton', 'Terre Haute', 'Auburn', 'Battle Creek', 'Hagerstown', 'Kennewick', 'Dover1', 'Lake Charles', 'May Pen', 'Port Charlotte', 'Mansfield', 'Baytown', 'Quillacollo', 'Slidell', 'San Fernando1', 'Morelia', 'Orizaba', 'Los Mochis', 'Lafayette2', 'Spanish Town', 'Longview1', 'New Albany', 'Sherbrooke', 'Johnson City', 'Kilowna', 'Puerto la Cruz', 'Bryan', 'New Bedford', 'Harlingen', 'Sangolqui', 'New London', 'Southaven', 'Panama City2', 'Winter Haven', 'Reynosa', 'Soledad', 'Aguascalientes', 'Vina del Mar', 'Saltillo', 'Alajuela', 'Rock Island', 'Beaver Falls', 'Miami Beach', 'Sarnia', 'Petersburg', 'Spring Hill', 'Independence', 'Waterbury', 'Lafayette1', 'Spartanburg', 'Elkhart', 'Appleton', 'Lancaster3', 'Poughkeepsie', 'London1', 'Cabimas', 'Tlaxcala', 'Kitchener', 'Metairie', 'Salem1', 'South Bend', 'Rockford', 'Barlett', 'McAllen', 'Waukesha', 'Oshawa', 'Sanford', 'Hampton', 'San Bernardo', 'Durham', 'Elgin', 'Melbourne1', 'Aurora1', 'Bridgeport', 'Kansas City2', 'Springfield4', 'Allentown', 'National City', 'Greensboro', 'Joliet', 'Pasadena1', 'Evanston', 'Grand Prairie1', 'Stamford', 'Coral Springs', 'Trenton', 'Glendale', 'Mesa', 'Long Beach', 'Windsor1', 'Nezahualcoyotl', 'Riverside', 'Marietta', 'Paterson', 'Baker Lake', 'Cape Dorset', 'Oxford House', 'Banff', 'Lansdowne House', 'Punta Prieta', 'Basse-terre', 'Bavaro', 'Chilca', 'Mitla', 'Nuqui', 'Papasquiaro', 'Toconao', 'Tonantins', 'Tula2', 'Grand Canyon', 'Rodeo', 'Fort Resolution', 'Milagro', 'Cuencame', 'Frontera', 'George Town1', 'Principe da Beira', 'Puerto Lempira', 'San Rafael2', 'Trujillo2', 'Tupiza', 'John Day', 'Mingan', 'Orocue', 'Tocache', 'Kimmirut', 'Las Lajas', 'Livingston', 'Los Blancos', 'Sixaola', 'Tombstone', 'Athabasca', 'Cobalt', 'Little Current', 'Puerto Natales', 'Shamattawa', 'Padilla', 'Apolo', 'Caballococha', 'El Dorado2', 'Huarmey', 'Punta Gorda', 'Santa Rita', 'Chapleau', 'Kanab', 'Kitty Hawk', 'Matagami', 'Mistassini', 'Wiarton', 'Aiquile', 'Barcelos', 'Cajabamba', 'El Manteco', 'Grand Turk', 'Jasper', 'La Cruz1', 'Lake Louise', 'Macas', 'Salaverry', 'Chos Malal', 'Monticello', 'Puerto San Julian', 'Nipigon', 'Carahue', 'Apalachicola', 'Ayaviri', 'Desaguadero', 'El Fuerte', 'Escuinapa', 'Huasco', 'Magdalena', 'Mejillones', 'Santa Barbara1', 'Taltal', 'Vallegrande', 'Zamora2', 'Biggar', 'Bishop', 'Digby', 'Gila Bend', 'Mobridge', 'Red Lake', 'Chota', 'Ciudad Cortes', 'Coalcoman', 'Glasgow1', 'La Paz2', 'Sechura', 'Bar Harbor', 'Hearst', 'Fort Chipewyan', 'Rankin Inlet', 'Aldama', 'Challapata', 'Contamana', 'Belen', 'Darregueira', 'Juan Aldama', 'Montpelier1', 'Parowan', 'Quellon', 'Cafayate', 'Caldera', 'Chachapoyas', 'Coracora', 'Gaspe', 'Halacho', 'La Cruz2', 'La Libertad', 'Macara', 'Novo Airao', 'Olmos', 'Socorro2', 'Tecuala', 'Cerrillos', 'Ely2', 'San Ramon', 'Tolu', 'Valparaiso1', 'Windsor2', 'Villa Ahumada', 'Willcox', 'Bluefields', 'Junin2', 'Perito Moreno', 'Puerto Carreno', 'Puerto Lopez', 'Quesada', 'San Javier', 'Tocopilla', 'Las Tablas', 'Lebu', 'Monte Cristi', 'San Martin2', 'Thermopolis', 'Tlaxiaco', 'Cochrane', 'Santa Rosalia', 'Santiago Tuxtla', 'Shelburne', 'Kindersley', 'Wawa', 'Salmon', 'Eldorado', 'Hudson Bay', 'Felipe Carrillo Puerto', 'Placetas', 'Sarmiento', 'Falfurrias', 'Huatabampo', 'Salamanca2', 'Compostela', 'Manicore', 'Victorica', 'Gunnison', 'San Justo', 'Victoria3', 'Chadron', 'Pijijiapan', 'The Pas', 'Chulucanas', 'Carhue', 'Las Lomitas', 'Huancavelica', 'Huanta', 'Juanjui', 'Paita', 'Tucumcari', 'Bermejo', 'Los Lagos', 'Sidney', 'Coari', 'Meadow Lake', 'Pamplona1', 'Turbo', 'Juticalpa', 'Petatlan', 'Steinbach', 'Fonte Boa', 'Pimentel', 'Puerto Berrio', 'Ayapel', 'Obando', 'Tamuin', 'Lamar', 'Filadelfia', 'Izamal', 'Vera', 'Manacapuru', 'Moyobamba', 'Pisco', 'Tunuyan', 'Charata', 'San Jose2', 'Chiquinquira', 'Atikokan', 'Banes', 'Chancay', 'Douglas2', 'Moab', 'Petoskey', 'Sicuani', 'Bathurst1', 'Bathurst2', 'Parry Sound', 'Frias', 'Libertador General San Martin', 'Sunchales', 'Arauca', 'Chamical', 'El Carmen de Bolivar', 'McCook', 'Pacasmayo', 'Antigonish', 'La Barca', 'Lota', 'Nueva Gerona', 'Tame', 'Cody', 'Tuxpan', 'Puerto Armuelles', 'Tenosique', 'Constitucion', 'Socorro1', 'Anatuya', 'Malargue', 'Puerto Cabezas', 'Villazon', 'Yarmouth', 'Champoton', 'Havre', 'Orange Walk', 'Raton', 'Tucupita', 'Dauphin', 'Green River', 'Moquegua', 'Rio Colorado', 'Trinidad1', 'Craig', 'Price', 'Ancud', 'Linares3', 'Bemidji', 'Ovalle', 'Cabo San Lucas', 'Juan Jose Castelli', 'Guymon', 'Pecos', 'Port Lavaca', 'Abancay', 'Colon2', 'Nelson2', 'Pochutla', 'Kenora', 'Sagua la Grande', 'San Felipe1', 'Illapel', 'Hancock', 'Revelstoke', 'Dumas', 'Los Andes', 'Caracarai', 'Liberia', 'Trujillo1', 'Winnemucca', 'Yankton', 'Chone', 'Mazatenango', 'Panuco', 'Parras', 'Rhinelander', 'Vallenar', 'Eirunepe', 'Needles', 'Rawlins', 'Talara', 'Brooks', 'Cayambe', 'Altagracia de Orituco', 'Tena', 'San Pedro de las Colonias', 'Williston', 'Ticul', 'Tingo Maria', 'Zaraza', 'Cananea', 'Orangeville', 'Rivas', 'Coffeyville', 'Fort Stockton', 'Nasca', 'Puerto Escondido', 'Sena Madureira', 'Granada1', 'Ciudad Camargo', 'Ilo', 'Jamestown2', 'Tumbes', 'Riverton', 'Moron1', 'Valladolid2', 'Mitchell', 'Swift Current', 'Belize City', 'Cadillac', 'Pendleton', 'Puno', 'Balboa', 'Matagalpa', 'El Banco', 'Guantanamo', 'Potosi', 'Brookings', 'Cienaga', 'Santiago1', 'Beeville', 'San Fernando2', 'Boulder City', 'Escanaba', 'Honda', 'Yorkton', 'Jipijapa', 'Flores', 'Quibdo', 'Dickinson', 'Tehuantepec', 'Cobija', 'San Carlos del Zulia', 'Vernal', 'Artemisa', 'Caborca', 'Duitama', 'Laramie', 'Manzanillo2', 'Coquimbo', 'Sancti Spiritus', 'Huaraz', 'Mont-Laurier', 'Rio Grande2', 'Cumberland', 'Los Alamos', 'Woodward', 'Cozumel', 'Ojinaga', 'Machiques', 'Penticton', 'Marquette', 'Bay City', 'San Pedro1', 'Lorica', 'Kingsville', 'Ciudad Guzman', 'General Pico', 'San Ignacio', 'Jaen', 'Agua Prieta', 'Upata', 'Guamuchil', 'Douglas1', 'Cedar City', 'Selma', "Val d'Or", 'Tulua', 'Poplar Bluff', 'Riohacha', 'Comayagua', 'Puerto Maldonado', 'Durango2', 'Ometepec', 'Ciudad del Carmen', 'Richland', 'Emporia', 'Aberdeen1', 'Dodge City', 'Viedma', 'Barahona', 'Florencia', 'Rio Tercero', 'Key West', 'Moose Jaw', 'Montrose', 'Ushuaia', 'Tefe', 'Calama', 'Alamogordo', 'Sullana', 'Leon1', 'North Platte', 'Moorhead', 'Ocana', 'Superior', 'Rolim de Moura', 'Norfolk2', 'Tunja', 'Choluteca', 'Carbondale', 'Nuevitas', 'Babahoyo', 'Chetumal', 'Venado Tuerto', 'Nuevo Casas Grandes', 'Matehuala', 'Scottsdale2', 'Gallup', 'Marinette', 'Edmundston', 'Huntsville1', 'Huanuco', 'Mason City', 'Curico', 'Camana', 'Scottsbluff', 'Watertown', 'Winona', 'Bayamo', 'Guanajuato', 'Cuauhtemoc', 'Ponca City', 'Atlantic City', 'Puntarenas', 'Junin1', 'Shawnee', 'Hobbs', 'Carlsbad', 'Las Tunas', 'Montero', 'Bozeman', 'Colon1', 'El Dorado1', 'Mercedes2', 'Zamora1', 'Del Rio', 'Saint-Georges1', 'Butte', 'El Centro', 'Enterprise', 'Popayan', 'Latacunga', 'Barcelona2', 'Burlington1', 'West Bend', 'Esmeraldas', 'Grand Prairie2', 'La Romana', 'Guajara-Miram', 'San Rafael1', 'Kalispell', 'Fredericton', 'Machala', 'Huehuetenango', 'Salina Cruz', 'Biloxi', 'Walla Walla', 'Chincha Alta', 'Gillette', 'Calabozo', 'Lethbridge', 'Rancagua', 'Gonaives', 'San Pedro de Macoris', 'Armenia', 'Monteria', 'Galveston', 'Chinandega', 'Rimouski', 'Red Deer', 'Fresnillo', 'Ithaca', 'Delicias', 'San Antonio2', 'Vicksburg', 'Chillan', 'Uruapan', 'Eagle Pass', 'Buenaventura', 'Tuxpam', 'Pergamino', 'Morgantown', 'Santa Marta', 'Chicoutimi', 'Dublin1', 'Sydney2', 'Matanzas', 'Natchez', 'Huajuapan de Leon', 'Greenville1', 'Medicine Hat', 'Ames', 'Escuintla', 'Christiansted', 'Clovis', 'Lagos de Moreno', 'Riobamba', 'Twin Falls', 'Navajoa', 'Victoriaville', 'Oranjestad', 'Ipiales', 'Concord', 'Sincelejo', 'Grand Forks', 'Jamestown1', 'Myrtle Beach', 'Pinar del Rio', 'Newport', 'Shawinigan', 'Loja', 'Fond du Lac', 'Cajamarca', 'Cordoba3', 'Tehuacan', 'Manta', 'Osorno', 'Carson City', 'Tupelo', 'Salina', 'Beaufort', 'Salamanca1', 'La Rioja', 'Piedras Negras', 'San Fernando de Apure', 'Sherman', 'Cornwall', 'Talca', 'Pativilca', 'Colima', 'Quincy', 'Frankfort', 'Cumana', 'Kingman', 'Tepic', 'Zacatecas', 'Coro', 'Santa Clara', 'Oak Ridge', 'Columbia1', 'Roswell', 'Traverse City', 'Puerto Vallarta', 'Pocatello', 'Dubuque', 'Lewiston1', 'Muskogee', 'Bangor', 'Garzon', 'Montego Bay', 'Poza Rica de Hidalgo', 'Hattiesburg', 'Port Arthur', 'Yuma', 'Conway', 'Logan', 'Chiclayo', 'Kankakee', 'Ibague', 'Temple', 'Danville', 'Brunswick', 'Bristol1', 'Prescott', 'Kokomo', 'Paducah', 'Lewiston2', 'State College', 'Guasave', 'Winchester', 'Elmira', 'Beckley', 'Gadsden', 'Pointe-a-Pitre', 'Santa Ana', 'Hot Springs', 'Boulder', 'Houma1', 'Oshkosh', 'Valdosta', 'Catamarca', 'Ibarra', 'Wheeling', 'Jonesboro', 'Lufkin', 'Maiquetia', 'Goldsboro', 'Ciudad Victoria', 'Pine Bluff', 'Aiken', 'Tamazunchale', 'Lazaro Cardenas', 'Bowling Green1', 'Ensenada', 'Duluth', 'Owensboro', 'Ciudad Obregon', 'Coatzacoalcos', 'Acarigua', 'Jefferson City', 'St. John', 'Fort-de-France', 'Bloomington1', 'Idaho Falls', 'Decatur', 'Arecibo', 'Sumter', 'Titusville', 'Johnstown', 'Visalia', 'Rapid City', 'Charlottesville', 'Jackson2', 'Tapachula', 'Monclova', 'Camaguey', 'Barrie', 'Lawton', 'Pereira', 'Sioux City', 'Maturin', 'Joplin', 'Norman', 'Wausau', 'Davenport', 'Florence1', 'Ponce', 'Rochester2', 'Greeneville', 'Council Bluffs', 'Porlamar', 'St. George', 'Santiago del Estero', 'Midland', 'Punto Fijo', 'Teziutlan', 'Bloomington2', 'Durango1', 'Lima1', 'Sudbury', 'Anderson', 'El Tigre', 'Dothan', 'San Angelo', 'St. Joseph', 'Willemstad', 'Texarkana', 'Muncie', 'St. Cloud', 'Urbana', 'Odessa1', 'Huntington', 'Pueblo', 'Ciudad Bolivar', 'Waterloo', 'Kingston2', 'Alton', 'San Pedro Sula', 'Alexandria1', 'Utica', 'Moncton', 'Abilene', 'Las Cruces', 'Fort Smith1', 'St. Augustine', 'Jacksonville2', 'Killeen', 'Wichita Falls', 'Muskegon', 'Rock Hill', 'Columbia2', 'Brownsville', 'Racine', 'Cap-Haitien', 'Tyler', 'Vero Beach', 'Grand Junction', 'Neiva', 'Caldwell', 'Gulfport', 'Cucuta', 'Murfreesboro', 'Springfield2', 'Beaumont', 'Ambato', 'Denton', 'Saginaw1', 'Pachuca', 'Portland1', 'Wilkes Barre', 'Monroe', 'Albany1', 'Athens1', 'Erie', 'Scranton', 'Tuscaloosa', 'Binghamton', 'Schenectady', 'Hickory', 'Villahermosa', 'Fayetteville2', 'Waco', 'Macon', 'Cochabamba', 'Cedar Rapids', 'Lynchburg', 'Niagara Falls', 'Fredericksburg', 'Ft. Myers', 'Queretaro', 'Les Cayes', 'Clarksville', 'Lubbock', 'Gainesville', 'Lincoln1', 'Cape Coral', 'Belleville1', 'Lancaster1', 'Evansville', 'Peoria', 'Harrisburg', 'Barquisimeto', 'Ocala', 'Green Bay', 'Kalamazoo', 'Fort Collins', 'Edinburg', 'Lexington', 'Manchester1', 'Virginia Beach', 'Wilmington1', 'Springfield1', 'York1', 'Asheville', 'Ann Arbor', 'Kissimmee', 'Provo', 'Daytona Beach', 'Madison', 'Roanoke', 'Lansing', 'Huntsville2', 'Bucaramanga', 'Mobile', 'Worcester1', 'Columbus1', 'Pensacola', 'Lakeville', 'Little Rock', 'Canton', 'Fort Pierce', 'Albany2', 'Bakersfield', 'Cuernavaca', 'Naples1', 'Palm Springs', 'Ogden', 'Fort Wayne', 'Fayetteville1', 'Chattanooga', 'Augusta1', 'Flint', 'Youngstown', 'Mayaguez', 'Ciudad Ju  rez', 'Winston-Salem', 'Wichita', 'Coral Gables', 'Charleston1', 'Oceanside', 'Toledo1', 'Sarasota', 'San Bernardino', 'New Haven', 'Greenville2', 'Covington', 'Zumpango', 'Toluca', 'Hamilton2', 'Gary', 'Grand Rapids', 'Aurora2', 'Waukegan', 'Akron', 'Dayton', 'Hartford', 'Fort Lauderdale', 'Rochester1', 'St. Petersburg1', 'Louisville', 'St. Charles1', 'Lowell', 'Birmingham1', 'West Palm Beach', 'Providence', 'Columbus2', 'Alexandria2', 'Charlotte', 'Irvine', 'Pasadena2', 'Pontiac', 'Newark', 'Arviat', 'Chesterfield Inlet', 'Taloyoak', 'Kuujjuarapik', 'Kuujjuaq', 'Fort Smith2', 'Qaanaq', 'Peace River', 'El Calafate', 'Labrador City', 'Thompson', 'Rawson', 'Fort McMurray', 'Puerto Limon', 'Yellowknife', 'Trinidad2', 'Coihaique', 'Tecoman', 'International Falls', 'Guaynas', 'Copiapo', 'Timmins', 'Elko', 'Augusta2', 'Brandon', 'La Serena', 'Huancayo', 'Tarija', 'Puerto Ayacucho', 'Arica', 'Campeche', 'Valparaiso2', 'Pasto', 'La Paz1', 'Los Angeles2', 'Minot', 'Santa Rosa2', 'David', 'North Bay', 'Flagstaff', 'Nuevo Laredo', 'Piura', 'Chimbote', 'Iquitos', 'Manizales', 'San Carlos de Bariloche', 'Missoula', 'Pucallpa', 'Great Falls', 'Freeport2', 'Casper', 'Nogales', 'Sault Ste. Marie', 'Santiago de Cuba', 'Cuenca', 'Neuquen', 'Santa Fe2', 'Fargo', 'Burlington2', 'Matamoros', 'Billings', 'Porto Velho', 'Salta', 'Torreon', 'Hermosillo', 'Barranquilla', 'Sioux Falls', 'Santa Barbara2', 'Laredo', 'Topeka', 'Arequipa', 'Charleston2', 'Amarillo', 'Culiacan', 'Corpus Christi', 'Leon2', 'Savannah', 'Santiago2', 'Mexicali', 'San Miguel de Tucuman', 'San Luis Potosi', 'Montgomery', 'Oaxaca', 'Wilmington2', 'San Juan2', 'Reno', 'Tallahassee', 'Shreveport', 'Spokane', 'El Paso', 'Syracuse', 'Rosario1', 'Knoxville', 'Fresno', 'Colorado Springs', 'Quetzaltenango', 'Baton Rouge', 'Columbia3', 'Tulsa', 'Albuquerque', 'St. Paul', 'Kansas City1', 'Tucson', 'Richmond1', 'Austin', 'Jacksonville1', 'Ft. Worth', 'Orlando', 'Baltimore', 'Indianapolis', 'Arctic Bay', 'Fort Severn', 'Nain', 'Puerto Deseado', 'Basseterre', 'Belmopan', 'Saint George ۪s', 'Santa Cruz1', 'Pierre', 'Roseau', 'Leticia', 'Castries', 'Kingstown', 'Riberalta', 'Cruzeiro do Sul', "Saint John's", 'Punta Arenas', 'Rio Gallegos', 'Charlottetown', 'Sucre', 'Iquique', 'Vilhena', 'Helena', 'Ica', 'Puerto Montt', 'Valdivia', 'Hamilton1', 'Tacna', 'Antofagasta', 'Ji-Parana', 'Bismarck', 'Cusco', 'Mazatlan', 'Cancun', 'Cheyenne', 'Santa Fe1', 'Ciudad Guayana', 'Trujillo3', 'Nassau', 'Regina', 'Tuxtla Gutierrez', 'Veracruz', 'Thunder Bay', 'Concepcion1', 'San Cristobal', 'Tegucigalpa', 'Boa Vista', 'Port-of-Spain', 'Comodoro Rivadavia', 'Saskatoon', 'Tampico', 'Kingston1', 'Managua', 'Acapulco', 'Bahia Blanca', 'Halifax', 'Panama City1', 'Cartagena1', 'Chihuahua', 'La Paz3', 'New Orleans', 'Medellin', 'Merida', 'Maracaibo', 'Caracas', 'Guayaquil', 'Cali', 'Manaus', 'Mendoza', 'Boise', 'San Jose1', 'Memphis', 'Jackson1', 'Winnipeg', 'Valencia1', 'Des Moines', 'Havana', 'Quito', 'Tijuana', 'Edmonton', 'Port-au-Prince', 'Cordoba1', 'San Salvador', 'Quebec', 'Omaha', 'Guatemala', 'Calgary', 'Santo Domingo', 'Puebla', 'Ottawa', 'San Diego', 'Norfolk1', 'Guadalajara1', 'Santiago3', 'Milwaukee', 'Buffalo', 'Monterrey', 'Oklahoma City', 'Raleigh', 'Phoenix', 'Salt Lake City', 'St. Louis', 'Nashville', 'Lima2', 'Montreal', 'Las Vegas2', 'Cincinnati', 'Detroit', 'Denver', 'San Antonio1', 'Tampa', 'Pittsburgh', 'Minneapolis', 'San Juan1', 'Cleveland', 'Dallas', 'Philadelphia', 'Houston', 'Miami', 'Bogota', 'Mexico City', 'Washington, D.C.', 'New York', 'Los Angeles1', 'Chicago', 'Boston', 'Toronto', 'Atlanta'])
#l3= list(['Port Hope Simpson', 'Punta del Este', 'Apodi', 'Lethem', 'Portel', 'Roura', 'Santa Cruz Cabralia', 'Yelimane', 'Baltasar Brum', 'Arroyos y Esteros', 'Barras', 'Koidu', 'Sinnamary', 'La Scie', 'Brownsweg', 'Koulikoro', 'Quixada', 'Santa Cruz2', 'Aigua', 'La Paloma', 'Saint-Laurent-du-Maroni', 'Tranqueras', 'Donegal', 'Angra do Heroismo', 'Vergara', 'Nova Vicosa', 'San Lorenzo1', 'Bogue', 'Caazapa', 'Paraguari', 'San Pedro2', 'Ypacarai|Ita', 'Selfoss', 'Tidjikdja', 'Ubaitaba', 'San Juan Bautista', 'Villa Hayes', 'Itapecuru Mirim', 'Comandante Fontana', 'Deer Lake', 'Keflavik', 'Coronel Bogado', 'Santa Lucia', 'Caninde', 'Diamantina', 'Artigas', 'Paso de los Toros', 'Shannon', 'Camocim', 'Campo Maior', 'Porto Nacional', 'Sao Lourenco do Sul', 'Rosario2', 'Obidos', 'Moengo', 'Jaguaquara', 'Dimbokro', 'Almenara', 'Iturama', 'Lapa', 'Sao Jose de Ribamar', 'Quarai', 'Akureyi', 'Santa Vitoria do Palmar', 'Kati', 'Maracaju', 'Markala', 'Cristalina', 'Guiglo', 'Bacabal', 'Sefra', 'Gualeguay', 'Brumado', 'Nanuque', 'Laguna', 'Arcoverde', 'Miracema', 'Itaberai', 'Juazeiro', 'San Jose de Mayo', 'Russas', 'Palmeira dos Indios', 'Lisburn', 'Treinta y Tres', 'Frutal', 'Rosario do Sul', 'Ceres', 'Linden', 'Omagh', 'Dolores', 'Jardim', 'Minas', 'Morrinhos', 'Registro', 'Tacuarembo', 'Linares2', 'Mamou', 'Osorio', 'Sobral', 'Goulimine', 'Sao Luiz Gonzaga', 'Castro2', 'Concepcion2', 'Braganca', 'Ksar El Kebir', 'Alagoinhas', 'Bawku', 'Cabo de Santo Agostinho', 'Capanema', 'Dabou', 'Balcarce', 'Tralee', 'Covilha', 'Zarate', 'Inhumas', 'Sao Gabriel', 'Ipora', 'Caratinga', 'Nova Lima', 'Concepcion del Uruguay', 'Jacarezinho', 'Mercedes1', 'Itapetinga', 'Joacaba', 'Winneba', 'Estancia', 'Goya', 'Coronel Oviedo', 'Canoinhas', 'Chivilcoy', 'Irati', 'Corner Brook', 'Nieuw Nickerie', 'Telemaco Borba', 'Bolgatanga', 'Garca', 'Campana', 'Carazinho', 'Batatais', 'Formiga', 'Koforidua', 'Pirassununga', 'Waterford', 'Ourense', 'Diourbel', 'Santo angelo', 'Macae', 'Cachoeira do Sul', 'San-Pedro', 'Cacador', 'Ararangua', 'Muriae', 'Guaxupe', 'Maldonado', 'Penapolis', 'Canela', 'Ituiutaba', 'Rio Verde2', 'Braganca Paulista', 'Rio Negro', 'Iguatu', 'Santana do Livramento', 'Timon', 'Catalao', 'Santa Ines', 'Itanhaem', 'Sidi bel Abbes', 'Sao Joao da Boa Vista', 'Tupa', 'Greenock', 'Itapetininga', 'Sao Joao del Rei', 'Erechim', 'Paranagua', 'Algeciras', 'Tres Lagoas', 'Korhogo', 'Passos', 'Jaboticabal', 'Araguari', 'Barretos', 'Tubarao', 'Tandil', 'Pirapora', 'Carlisle', 'El Jadida', 'Avare', 'Ponta Pora', 'Arapongas', 'Apucarana', 'Itabuna', 'Guarapuava', 'Conselheiro Lafaiete', 'Vitoria2', 'Chester', 'Santa Cruz do Sul', 'Itauna', 'Cachoeiro de Itapemirim', 'Soubre', 'Barra Mansa', 'Barbacena', 'Tatui', 'Brusque', 'Xapeco', 'Tarbes', 'Thies', 'St.-Brieuc', 'Nova Friburgo', 'Gijon', 'Ourinhos', 'Leon3', 'Aracatuba', 'Setubal', 'Pocos de Caldas', 'Jaragua do Sul', 'Sete Lagoas', 'Governador Valadares', 'Dourados', 'Pouso Alegre', 'Juazeiro do Norte', 'Corrientes', 'Sekondi', 'Passo Fundo', 'Catanduva', 'Trindade', 'Marbella', 'Santa Maria2', 'Sao Carlos', 'Cascavel', 'Presidente Prudente', 'Piracicaba', 'Marilia', 'Rio Claro', 'Brikama', 'Encarnacion', 'Juiz de Fora', 'Varzea Grande', 'Pindamonhangaba', 'Maringa', 'Angers', 'Sao Jose do Rio Preto', 'Biarritz', 'Blumenau', 'Olinda', 'Meknes', 'Guaratingueta', 'Joinville', 'Southampton', 'Campina Grande', 'Cabo Frio', 'Jundiai', 'Sunderland', 'Carpina', 'Rio Largo', 'Murcia', 'Swansea', 'Tema', 'Bournemouth', 'Leicester', 'Bristol2', 'Sao Jose dos Campos', 'Jaboatao', 'Kumasi', 'Itu', 'Sao Jose dos Pinhais', 'Leeds', 'Vitoria1', 'Canoas', 'Americana', 'Niteroi', 'Duque de Caxias', 'Bradford', 'San Lorenzo2', 'Reading', 'Sheffield', 'Campinas', 'Nova Iguacu', 'Braga', 'Santo Andre', 'Rigolet', 'St. Anthony', 'Trout River', 'Bansang', 'Cameta', 'Iracoubo', 'Saint-Georges2', 'Satadougou', 'Selibaby', 'Timbedra', 'Trepassey', 'Tucano', 'Tucurui', 'Viseu', 'Chevery', 'Boke', 'Mato Grosso', 'Cartwright', 'Colinas', 'Ipu', 'Banamba', 'La Victoria', 'Rosario3', 'Santa Maria da Vitoria', 'Sokolo', 'Buchans', 'Saudarkrokur', 'Isafjordur', 'Abai', 'Amapa', 'Paracuru', 'Robertsport', 'Totness', 'Xinguara', 'Argentia', 'Acarau', 'Baturite', 'Egilsstadir', 'Granja', 'Harper', 'Xique-Xique', 'Channel-Port aux Basques', 'Lascano', 'Beni Ounif', 'Itapipoca', 'Viana', 'Yomou', 'Aracati', 'Goundam', 'Itambe', 'Anna Regina', 'Nioro du Sahel', 'Greenville3', 'Kirkwall', 'Acu', 'Bartica', 'Boutilimit', 'Salgueiro', 'Valenca', 'Bolama', 'Altamira', 'Ico', 'Tocantinopolis', 'Lerwick', 'Kerouane', 'Faranah', 'Itaberaba', 'Kaya', 'Salinopolis', 'Senhor do Bonfim', 'Bella Union', 'Gander', 'Horqueta', 'Chapadinha', 'Mabaruma', 'Bondoukou', 'Juina', 'Kedougou', 'Kourou', 'Maues', 'Odienne', 'Carmelo', 'Banfora', 'Breves', 'Iguape', 'Imbituba', 'Caico', 'Horta', 'Gbarnga', 'Bocaiuva', 'Zouirat', 'Seguela', 'Yendi', 'Kiffa', 'Robore', 'Abaetetuba', 'Juarez', 'Zwedru', 'Oriximina', 'Sassandra', 'Guanhaes', 'Pinheiro', 'Ita', 'Killarney', 'Sao Francisco do Sul', 'Smara', 'Thorshavn', 'Aracuai', 'Bougouni', 'Crateus', 'Siguiri', 'Wa', 'Novo Horizonte', 'Kindia', 'Niquelandia', 'Cadiz1', 'Cadiz2', 'Januaria', 'Barra do Bugres', 'Kita', 'Ferkessedougou', 'Palmas', 'Rosso', 'Jaguarao', 'Leopoldina', 'Nara', 'Sligo', 'Paragominas', 'Pilar', 'Guaira', 'Itacoatiara', 'Mercedes4', 'Pires do Rio', 'Pontes e Lacerda', 'San', 'Nkawkaw', 'Ad Dakhla', 'Berekum', 'Nova Cruz', 'Stephenville', 'Picos', 'Irece', 'Pedro Juan Caballero', 'Durazno', 'Araguaina', 'Balsas', 'Fria', 'Grand Bassam', 'Mopti', 'Kilkenny', 'Bella Vista', 'Penzance', 'Douglas3', 'Porto Santana', 'Larache', 'Piripiri', 'Lobos', 'Mindelo', 'Mineiros', 'Drogheda', 'New Amsterdam', 'Makeni', 'Tambacounda', 'Floriano', 'Macenta', 'Sinop', 'Tenkodogo', 'Tindouf', 'Ilheus', 'Louga', 'Sikasso', 'Concordia2', 'Itaituba', 'Kolda', 'Ouahigouya', 'Paranaiba', 'Tan Tan', 'Jequie', 'Porto Seguro', 'Bafata', 'Arapiraca', 'Buchanan', 'Chascomus', 'Tiznit', 'Itamaraju', 'Kamsar', 'Melo', 'Faro', 'Lorca', 'Guadalajara2', 'Goianesia', 'Camaqua', 'Curvelo', 'Janauba', 'Dundalk', 'Rivera', 'Gualeguaychu', 'Perth1', 'Gurupi', 'Kenema', 'Koutiala', 'Campo Belo', 'Sao Borja', 'Paracatu', 'Dumfries', 'Ja̩n', 'Kissidougou', 'Paysandu', 'Sao Mateus', 'Inverness', 'Santiago de Compostela', 'Taza', 'Vacaria', 'Toledo2', 'Mar de Ajo', 'Penedo', 'Arrecife', 'Gibraltar', 'Lujan', 'Barra do Garcas', 'Adrar', 'Settat', 'Ayr1', 'Teofilo Otoni', 'Andradina', 'Azul', 'Salto', 'Parnaiba', 'Guanambi', 'Ijui', 'Agboville', 'Huelva', 'Ziguinchor', 'Araxa', 'Formosa2', 'Saint-Louis', 'Daloa', 'Ouezzane', 'Scarborough', 'Labe', 'Santa Rosa3', 'Jatai', 'Reconquista', 'Linhares', 'Concordia1', 'Bo', 'Aquidauana', 'Ponte Nova', 'Caruaru', 'Koudougou', 'Man', 'Londonderry', 'Melilla', 'Albacete', 'Lavras', 'Almeria', 'Serrinha', 'Patos', 'Pedreiras', 'Campo Murao', 'Castanhal', 'Galway', 'Fernandopolis', 'Porto Uniao', 'Logrono', 'Mossoro', 'Gagnoa', 'Abengourou', 'Garanhuns', 'Necochea', 'Bath', 'Itapeva', 'Itumbiara', 'Bento Goncalves', 'Crato', 'Uruguaiana', 'Er Rachidia', 'Badajoz', 'Botucatu', 'Cherbourg', 'Burgos', 'Bage', 'Rio Grande1', 'Rondonopolis', 'Cartagena2', 'Exeter', 'Limerick', 'Uba', 'Lajes', 'Portimao', 'Salamanca3', 'Coimbra', 'La Rochelle', 'Vila Velha', 'Cape Coast', 'Dundee', 'Funchal', 'Jau', 'Tlimcen', 'Assis', 'York2', 'Bechar', 'Cork', 'Valladolid1', 'Lorient', 'Imperatriz', 'Tamale', 'Oviedo', 'Santa Cruz de Tenerife', 'Limeira', 'Malaga', 'Timbauba', 'Aberdeen3', 'Luton', 'Castello', 'Uberaba', 'Barreiros', 'Obuasi', 'Plymouth', 'Kenitra', 'Criciuma', 'Pamplona2', 'Petropolis', 'Rennes', 'Campos', 'Blackpool', 'Oxford', 'Itajai', 'Santander', 'Divinopolis', 'Brest1', 'Franca', 'Taubate', 'Zaragoza', 'Anapolis', 'San Sebasti  n', 'Granada2', 'Kingston upon Hull', 'Resistencia', 'Volta Redonda', 'Agadir', 'Alicante', 'Foz do Iguacu', 'La Coruna', 'Ipatinga', 'Tangier', 'Caen', 'Cuiaba', 'Coventry', 'Goiana', 'Brighton', 'Portsmouth', 'Edinburgh', 'Stoke', 'Joao Pessoa', 'Londrina', 'Aracaju', 'Middlesbrough', 'Novo Hamburgo', 'Nantes', 'Cardiff', 'Santos', 'La Plata', 'Nottingham', 'Liverpool2', 'Porto', 'Manchester2', 'Narsarsuaq', 'Alvorada', 'Ayounel el Atrous', 'Stanley', 'Kaedi', 'Mercedes3', 'Barreiras', 'Rocha', 'Nouadhibou', 'Segou', 'Gao', 'Petrolina', 'Sao Luis', 'Laayoune', 'Kayes2', 'Ciudad del Este', 'Ponta Delgada', 'Kankan', 'Corumba', 'Nzerekore', 'Kaolack', 'Safi', 'Las Palmas', 'Yamoussoukro', 'Vitoria da Conquista', 'Formosa1', 'Oujda', 'Bobo Dioulasso', 'Pelotas', 'Montes Claros', 'Bouake', 'Cordoba2', 'Vigo', 'Ponta Grossa', 'Bauru', 'Caxias do Sul', 'Teresina', 'Maceio', 'Uberlandia', 'Ribeirao Preto', 'Bilbao', 'Belfast', 'Sorocaba', 'Natal', 'Bordeaux', 'Belo Horizonte', 'Birmingham2', 'Macapa', 'Djenne', 'Banjul', 'Atar', 'Timbuktu', 'Caxias', 'Maraba', 'Posadas', 'Cayenne', 'Florianopolis', 'Praia', 'Georgetown1', 'Santarem', 'Bissau', 'Reykjavik', 'Freetown', 'Nouakchott', 'Feira de Santana', 'St. John ۪s', 'Paramaribo', 'Asuncion', 'Bridgetown', 'Fez', 'Monrovia', 'Bamako', 'Recife', 'Newcastle', 'Conakry', 'Mar del Plata', 'Marrakesh', 'Belem', 'Ouagadougou', 'Manaus', 'Campo Grande', 'Seville', 'Dakar', 'Porto Alegre', 'Oran', 'Valencia2', 'Montevideo', 'Salvador', 'Rabat', 'Casablanca', 'Brasilia', 'Accra', 'Glasgow2', 'Abidjan', 'Lisbon', 'Fortaleza', 'Curitiba', 'Goiania', 'Rio de Janeiro', 'Dublin2', 'Madrid', 'Sao Paolo', 'Buenos Aires', 'London2'])
#l4= list(['Al Jaghbub', 'Dirj', 'Gubkin', 'Bertoua', 'Dibaya', 'Fada', 'Manyoni', 'Nimule', 'Pala', 'Panda', 'Mandali', 'Besalampy', 'Manja', 'Plumtree', 'Kumo', 'Luebo', 'Celeken', 'Chibia', 'El Qasr', 'Geita', 'Ondjiva', 'Zelenodolsk', 'Finnsnes', 'Buurhakaba', 'Dondo2', 'Kambove', 'Karasburg', 'Kasane', 'Maridi', 'Mpwapwa', 'Oum Hadjer', 'Bosobolo', 'Eseka', 'Libenge', 'Masindi-Port', 'Bossembele', 'Bekiy', 'Babati', 'Kafue', 'Oshikango', 'Knysna', 'Illizi', 'Ambanja', 'Maqat', 'Katima Mulilo', 'Reggane', 'El Daba', 'Edea', 'Ali Sabih', 'Musina', 'Swellendam', 'Kimpese', 'Nikolayevsk', 'Izaz', 'Doba', 'Agordat', 'Demba', 'Yangambi', 'Alayat Samail', 'Polyarnyy', 'Jijiga', 'Businga', 'Hakkari', 'Kirsanov', 'St.-Benoit', 'Aliwal North', 'Bakal', 'Waku Kungo', 'Hermanus', 'Tsumeb', 'Khromtau', 'Mao', 'Swakopmund', 'Soyo', 'Ambatondrazaka', 'Slantsy', 'Mongbwalu', 'Ash Shihr', 'Maroantsetra', 'Al Khums', 'Mossel Bay', 'Gimbi', 'Ngara', 'Dawmat al Jandal', 'Novomoskovsk', 'Zheleznogorsk', 'Okahandja', 'Gyzlarbat', 'Boma', 'Rehoboth', 'Tatvan', 'Al Ahmadi', 'Ahar', 'Kericho', 'Mandera', 'Tskhinvali', 'Chitipa', 'Tessenei', 'Al Quwayiyah', 'Luleburgaz', 'Chur', 'Sennar', 'Balyqshy', 'EdDamer', 'Kabalo', 'Mugla', 'Zvishavane', 'Standerton', 'Kavala', 'Brovary', 'Gogrial', 'Uglich', 'Shahrud', 'Aketi', 'Al Marj', 'Nkongsamba', 'Foumban', 'Yevlax', 'Umm al Qaywayn', 'Vratsa', 'Tzaneen', 'Vlore', 'Lobito', 'Moanda2', 'Jamaame', 'Pyatigorsk', 'Umm Ruwaba', 'Severomorsk', 'Polatli', 'Machakos', 'Kapiri Mposhi', 'Alexandroupoli', 'Buguruslan', 'Birni Nkonni', 'Buzmeyin', 'Chililabombwe', 'Muglad', 'Novozybkov', 'Brits', 'Vryheid', 'Dyatkovo', 'Bethal', 'Nekemte', 'Uvira', 'Kohtla-Jarve', 'El Kharga', 'Shebekino', 'Gjovik', 'Klin', 'Kirovo-Chepetsk', 'Mo i Rana', 'Bani Walid', 'Gandajika', 'Berdyansk', 'Kamensk Shakhtinskiy', 'Hurghada', 'Tulcea', 'Kasangulu', 'As Salt', 'Berber', 'Varamin', 'Eldama Ravine', 'Haugesund', 'Laghouat', 'Kropotkin', 'Madinat ath Thawrah', 'Rzhev', 'Barletta', 'Chernyakhovsk', 'Al Qamishli', 'Koneurgench', 'Shumen', 'Apatity', 'Marzuq', 'Kanye', 'Middelburg2', 'Sliven', 'Rezekne', 'Nusaybin', 'Ammochostos', 'Vyshnniy Volochek', 'Drobeta-Turnu Severin', 'Severouralsk', 'Leninogorsk', 'Beja', 'Viborg', 'El Arish', 'Loubomo', 'Numan', 'Biel', 'Ruzayevka', 'Trnava', 'Asti', 'Crotone', 'Mozdok', 'Apsheronsk', 'Shchekino', 'Kizel', 'Manbij', 'Maladzyechna', 'Savonlinna', 'Oudtshoorn', 'Novoshakhtinsk', 'Roskilde', 'Balakhna', 'Oktyabrskiy', 'Borisoglebsk', 'Ben Gardane', 'Uman', 'Aleksin', 'Nanyuki', 'Vibo Valentia', 'Shuya', 'Prizren', 'Ash Shatrah', 'Elbasan', 'Kahama', 'Georgievsk', 'Pernik', 'Karlovac', 'Vanadzor', 'Illichivsk', 'Orekhovo-Zuevo', 'Idlib', 'Kindu', 'Elblag', 'Ragusa', 'El Manaqil', 'Zenica', 'Al Musayyib', 'Al Hasakah', 'Souk Ahras', 'Liepaga', 'Novara', 'Suceava', 'Masjed Soleyman', 'Ceske Budejovice', 'Nkhata Bay', 'Emden', 'Mafetang', 'Tiarat', 'Kislovodsk', 'Tebessa', 'Novotroitsk', 'Michurinsk', 'Mityana', 'Tetovo', 'Zrenjanin', 'Gyumri', 'Qal at Bishah', 'Banska Bystrica', 'Polatsk', 'Subotica', 'Khujayli', 'Slupsk', 'Inowroclaw', 'Buzau', 'Ukhta', 'Bila Tserkva', 'Baranavichy', 'Sibiu', 'Zanjan', 'Sanandaj', 'Szombathely', 'Kamyanets-Podilskyy', 'Al Jahra', 'Gafsa', 'Sassari', 'Isparta', 'Ad Diwaniyah', 'Neyshabur', 'Bugulma', 'Panevezy', 'Yanbu al Bahr', 'Az Zawiyah', 'Kecskemet', 'Wloclawek', 'Bataysk', 'Bojnurd', 'Kutahya', 'Volos', 'Kashmar', 'Balakovo', 'Kolomna', 'Arzamas', 'Olomouc', 'Novocherkassk', 'Ornskoldsvik', 'Siauliai', 'Gorzow Wielkopolski', 'Coburg', 'Sergiyev Posad', 'Keffi', 'Corum', 'Al Mubarraz', 'Liberec', 'Borlange', 'Szekesfehervar', 'Konotop', 'Sumqayt', 'Ouargla', 'Elazig', 'Arras', 'Beer Sheva', 'Svetogorsk', 'Hania', 'Plovdiv', 'Arad', 'Karabuk', 'Uzhgorod', 'Pakwach', 'Yelets', 'Hagere Hiywet', 'Ar Ramadi', 'Drohobych', 'Mostaganem', 'Sanliurfa', 'Solikamsk', 'Galati', 'Kragujevac', 'Slavonski Brod', 'Walbrzych', 'Khmelnytskyy', 'Kasulu', 'Wiener', 'Maykop', 'Az Aubayr', 'Kineshma', 'Reggio di Calabria', 'Tarbes', 'Chapayevsk', 'Al-Qatif', 'Oradea', 'Nyiregyhaza', 'Mykolayiv', 'Kahramanmaras', 'Roanne', 'Aix-en-Provence', 'Olsztyn', 'Taganrog', 'Najran', 'Qairouan', 'Pitesti', 'Przemysl', 'Nevinnomyssk', 'Mataro', 'Patra', 'Skien', 'Chernihiv', 'Ganca', 'Salalah', 'Setif', 'Welkom', 'Modena', 'Uitenhage', 'Pori', 'Rijeka', 'Kitwe', 'Curepipe', 'Poitier', 'Isna', 'Sterlitamak', 'Besancon', 'Paarl', 'Novokuybishevsk', 'Kramatorsk', 'Calabar', 'Kom Ombo', 'Erfurt', 'Vasteraas', 'Annecy', 'Jonkoping', 'Opole', 'Sotik', 'At Taif', 'Perpignan', 'Tambov', 'Rustenburg', 'Shinyanga', 'Lahti', 'Bhisho', 'Kolpino', 'Az Zarqa', 'Orumiyeh', 'Oldenburg', 'Jinja', 'Homyel', 'Furth', 'Al Khalil', 'Syzran', 'Aydin', 'Adapazari', 'Makhachkala', 'Udine', 'Kutaisi', 'Braunschweig', 'Dhamar', 'Springs', 'Matola', 'Makiyivka', 'Saida1', 'Samalut', 'El Minya', 'Al Ladhiqiyah', 'Az Zahran', 'Ondo', 'Eindhoven', 'Naberezhnyye Chelny', 'Icel', 'Oshogbo', 'Duma', 'Clermont-Ferrand', 'Kassel', 'Bielefeld', 'Augsburg', 'Treviso', 'Vereeniging', 'Pirai̩vs', 'Chemnitz', 'Horlivka', 'Ife', 'Damanh  r', 'Oyo', 'Kryvyy Rih', 'Tema', 'Melun', 'Strasbourg', 'Heidelberg', 'Karlsruhe', 'Sharjah', 'Kumbo', 'Tarnow', 'Dumyat', 'Bytom', 'Adi Ugri', 'El Mansura', 'Irbid', 'Antsirabe', 'Southend', 'Rize', 'Wuppertal', 'Benoni', 'Caserta', 'Gent', 'Duisburg', 'Liege', 'Onitsha', 'Lille', 'Girga', 'Essen', 'Shashemene', 'Nazret', 'Awka', 'Cyangugu', 'Debre Birhan', 'Nyanza', 'Como', 'Dortmund', 'Bafoussam', 'Ouidah', 'Ibb', 'Kisii', 'Nsukka', 'Ruhengeri', 'Kakamega', 'Bungoma', 'Gitarama', 'Ado Ekiti', 'Yirga Alem', 'Butare', 'Gitega', 'Al Bayda', 'Brandfort', 'I-n-Amenas', 'Port Alfred', 'Ubomba', 'Androka', 'Bagamoyo', 'Belabo', 'Cacolo', 'Calucinga', 'Chinsali', 'Chiramba', 'Djanet', 'Espungabera', 'Eyl', 'Ikela', 'Kaabong', 'Kalima', 'Kanyato', 'Kaoma', 'Kariba', 'Karungu', 'Katwe', 'Kenge', 'Kibiti', 'Komatipoort', 'Lamu', 'Lubutu', 'Mapai', 'Mirbat', 'Morondava', 'Moroto', 'Nata', 'Nyimba', 'Ouadda', 'Quipungo', 'Quissico', 'Sesheke', 'Tsavo', 'Tunduru', 'Vilanculos', 'Witu', 'Zouar', 'Kirkenes', 'Mizdah', 'Mtsensk', 'Vanhynsdorp', 'Kaniama', 'Poffader', 'Cangamba', 'Karoi', 'Lebowakgomo', 'Quibala', 'Abong Mbang', 'Basoko', 'Bayghanin', 'Birao', 'Bongandanga', 'Calulo', 'Fort Shevchenko', 'Kampene', 'Kontcha', 'Makhambet', 'Mubende', 'Namanga', 'Same', 'Soroti', 'Ayorou', 'Beitbridge', 'Bethanie', 'Biltine', 'Bol', 'Chibemba', 'Dehibat', 'Gwanda', 'Oranjemund', 'Kuruman', 'Bredasdorp', 'Camabatela', 'Caxito', 'Ghadamis', 'Hadiboh', 'Luau', 'Lusanga', 'Maintirano', 'Mandritsara', 'Mangai', 'Mbe', 'Mongo', 'Muconda', 'Ndende', 'Obock', 'Qaminis', 'Rumbek', 'Sikonge', 'Sumbe', 'Usakos', 'Antsohihy', 'Ati', 'Bukama', 'El Alamein', 'Garoowe', 'Itigi', 'Mananjary', 'Mbaiki', 'Miandrivazo', 'Ndalatando', 'Qasr-e Shirin', 'Tshela', 'Zambezi', 'Ermoupoli', 'Salum', 'Beledweyne', 'Chapaev', 'Farafangana', 'Ihosy', 'Lundazi', 'Melut', 'Merowe', 'Moloundou', 'Mwinilunga', 'Nguigmi', 'Nzeto', 'Sibiti', 'Uige', 'Yei', 'Bur Safaga', 'Colesberg', 'Mezen', 'Sitia', 'Boende', 'Cubal', 'Embi', 'Marovoay', 'Mekambo', 'Mpika', 'Tibati', 'Aqtau', 'Mitzik', 'Naryan Mar', 'El Tur', 'Lusambo', 'Tshabong', 'Bloemhof', 'Boosaaso', 'Gamba', 'Goure', 'Kitgum', 'Kribi', 'Macia', 'Madaoua', 'Moba', 'Mocambique', 'Mongomo', 'Golela', 'Gubakha', 'Sovetsk2', 'Dombarovskiy', 'Kirovsk', 'Lubao', 'Zongo', 'Camacupa', 'Dzaoudzi', 'Kos', 'Lucapa', 'Mushie', 'Prieska', 'Ghat', 'Bafia', 'Dembi Dolo', 'Voi', 'Al Qunaytirah', 'Graaff Reinet', 'Keetmanshoop', 'Outjo', 'Timimoun', 'Lgov', 'Mwingi', 'Nchelenge', 'Nzega', 'Velikiy Ustyug', 'Bolobo', 'Dilolo', 'Djambala', 'Gashua', 'Gaya1', 'Ghanzi', 'Aweil', 'Bongor', 'Kapoeta', 'Kasongo-Lunda', 'Mbanza-Congo', 'Shamva', 'Songo', 'Sparti', 'Tillaberi', 'Antalaha', 'As Sulayyil', 'Gharyan', 'Tolanaro', 'Kaga Bandoro', 'Lindi', 'Mansa', 'Nagele', 'Nalut', 'Ar Rutbah', 'Mednogorsk', 'Ongwediva', 'Sosnogorsk', 'Nikel', 'Asosa', 'Diffa', 'Gbadolite', 'Karema', 'Kelo', 'I-n-Salah', 'Oktyabrsk', 'Zhanibek', 'Andoany', 'Jalingo', 'Kasaji', 'Koulamoutou', 'Mbala', 'Mumbwa', 'Mwenga', 'Opobo', 'Sembe', 'Wukari', 'Al Wajh', 'Birak', 'Chiredzi', 'Volksrust', 'Kilindoni', 'Pallasovka', 'Goba', 'Koko', 'Bati', 'Krasnoarmeysk', 'Mariental', 'Victoria Falls', 'Aqsay', 'Kapan', 'Sibut', 'Bijar', 'Gay', 'Rundu', 'Jawhar', 'Makokou', 'Mouila', 'Tadmur', 'Yendi', 'Lokoja', 'Mbulu', 'Wum', 'Volgodonsk', 'Sodankyla', 'Port Shepstone', 'Segezha', 'Ulundi', 'Am Timan', 'Ewo', 'Kipushi', 'Hun', 'Hwange', 'Bria', 'Goycay', 'Watsa', 'Basankusu', 'Rayevskiy', 'Kirs', 'Kondopoga', 'Atkarsk', 'Shalqar', 'Tokar', 'Algha', 'Bozoum', 'Inongo', 'Kyustendil', 'Meiganga', 'Mweka', 'Biu', 'Chinhoyi', 'Mbalmayo', 'Nyunzu', 'Pechora', 'Kumertau', 'Kasama', 'Marrupa', 'Kamenka', 'Saldanha', 'Steinkjer', 'Gaalkacyo', 'Dosso', 'Bulungu', 'De Aar', 'Hassi Messaoud', 'Idah', 'Ilebo', 'Kirov1', 'Kotovsk', 'Marsabit', 'Mossendjo', 'Uvinza', 'Wadi Halfa', 'El Bayadh', 'El Golea', 'Kudymkar', 'Luga', 'Babanusa', 'Mazabuka', 'Kasimov', 'Moyale', 'Njombe', 'Tripoli2', 'Mitilini', 'Rostov2', 'Nicuadala', 'Kandi', 'Kismaayo', 'Ouesso', 'Owando', 'Nola', 'Kaspiysk', 'Makoua', 'Bitam', 'Siwa', 'Moyo', 'Naivasha', 'Sasovo', 'Solwezi', 'Sur', 'Bogoroditsk', 'Kabinda', 'Belebey', 'Sambava', 'Sovetsk1', 'Nelidovo', 'Kongolo', 'Palapye', 'Sarh', 'Wajir', 'Beaufort West', 'Baydhabo', 'Kawambwa', 'Nguru', 'Otjiwarongo', 'Owo', 'Solnechnogorsk', 'Carnot', 'Dundo', 'Bondo', 'Dolo Bay', 'Velsk', 'Djougou', 'Saurimo', 'Ad Nabk', 'El Kef', 'Masasi', 'Millerovo', 'Natitingou', 'Buy', 'Chusovoy', 'Singida', 'Yegoryevsk', 'Obninsk', 'Kontagora', 'Manono', 'Mikumi', 'Tikhvin', 'En Nuhud', 'Bama', 'Mahabad', 'Oyem', 'Ajdabiya', 'Cahul', 'Shumerlya', 'Blagodarnyy', 'Yeysk', 'Seres', 'Alatyr', 'Buynaksk', 'Darnah', 'Mardin', 'Amasya', 'Torzhok', 'Vac', 'Bologoye', 'Kakonko', 'Onega', 'Masvingo', 'Karpinsk', 'Kilis', 'Luena', 'Shkoder', 'Zahle', 'Alekseyevka', 'Khaskovo', 'Saywun', 'Vryburg', 'Vyazma', 'Bezhetsk', 'Frederikshavn', 'Mango', 'Embu', 'Slobodskoy', 'Gelendzhik', 'Biharamulo', 'Kotelnich', 'Garissa', 'Ngaoundere', 'Mikhaylovka', 'Stepanakert', 'Kaka', 'Yalta', 'Uryupinsk', 'Pugachev', 'Rafha', 'Montepuez', 'Saveh', 'Tuymazy', 'Akhtubinsk', 'Ali Bayramli', 'Birsk', 'Katerini', 'Visby', 'Tekirdag', 'Qulsary', 'Tunduma', 'Bodo', 'Kiruna', 'Valuyki', 'Dobrich', 'Tuapse', 'Ed Dueim', 'Tozeur', 'Wamba', 'Chistopol', 'Marsala', 'Inhambane', 'Iwo', 'Jihlava', 'Kadoma', 'Massawa', 'Rodos', 'Bafang', 'Bitola', 'Qasserine', 'Sol-lletsk', 'Maun', 'Sibay', 'Elk', 'Nizwa', 'Dubrovnik', 'Dondo1', 'Salsk', 'Bangassou', 'Buta', 'Tall Afar', 'Queenstown1', 'Karonga', 'Saki', 'Songea', 'Omutninsk', 'Staraya Russa', 'Roslavl', 'Gryazi', 'Krasnokamsk', 'Ventspils', 'Dzhankoy', 'Molde', 'Ho', 'Focsani', 'Kanash', 'Pula', 'Timashevsk', 'Lambarene', 'Azare', 'Batouri', 'Zvolen', 'Yefremov', 'Ifakara', 'Nelspruit', 'Serdobsk', 'Mozhga', 'Lobatse', 'Elista', 'Bollnas', 'Stara Zagora', 'Eyumojok', 'Pirgos', 'Civitavecchia', 'Kwekwe', 'Aksum', 'Safonovo', 'Rida', 'Cabinda', 'Lida', 'Otradnyy', 'Sokode', 'Siena', 'Burco', 'Kamina', 'Fasa', 'Kotlas', 'Mazyr', 'Surt', 'Thika', 'Frolovo', 'Derbent', 'Sharya', 'Lisala', 'Livny', 'Lodwar', 'Angoche', 'Vyborg', 'Harstad', 'Canakkale', 'Arar', 'Al Fallujah', 'Dover2', 'Grahamstown', 'Kuito', 'Hios', 'Neftekamsk', 'Vichuga', 'Iseyin', 'Bouar', 'Oturkpo', 'Sokol', 'Kandalaksha', 'Zelenokumsk', 'As Suwayda', 'Balkanabat', 'Sumbawanga', 'Agri', 'Yevpatoriya', 'Tubruq', 'Asela', 'Moss', 'Poti', 'Kimry', 'Turnovo', 'Potiskum', 'Yambio', 'Borovichi', 'Lillehammer', 'Bossangoa', 'Bethlehem', 'Behbehan', 'Jelgava', 'Kamyshin', 'Boorama', 'Ilam', 'Suwalki', 'Al Mukalla', 'Resita', 'Debre Markos', 'Asha', 'Morshansk', 'Zakho', 'Botosani', 'Porvoo', 'Kemi', 'Tikhoretsk', 'Kuznetsk', 'Klintsy', 'Kungur', 'Korosten', 'Olbia', 'Voznesensk', 'Maragheh', 'Lankaran', 'Prijedor', 'Kars', 'Pleven', 'Tikrit', 'Mus', 'Volsk', 'Svendborg', 'Choma', 'Jizan', 'Lomza', 'Bata', 'Krasnoufimsk', 'Qomsheh', 'Xanthi', 'Limbe', 'Ed Damazin', 'Semnan', 'Zilina', 'Kerch', 'Ladysmith', 'Salavat', 'Belaya Kalitva', "L'Aquila", 'Mongu', 'Naxcivan', 'Al Fujayrah', 'Gatchina', 'Parnu', 'Narva', 'Edirne', 'Iringa', 'Sandnes', 'Foggia', 'Auxerre', 'Zadar', 'Burgas', 'Matruh', 'Vyska', 'Kerkira', 'Shostka', 'Kristianstad', 'San Marino', 'Buzuluk', 'Sabzewar', 'Atbara', 'Volkhov', 'Usak', 'Arba Minch', 'Malindi', 'Mbanza-Ngungu', 'Benevento', 'Votkinsk', 'Bunia', 'Agrinio', 'Chake Chake', 'Monchegorsk', 'Chimboy', 'Sarapul', 'Sukhumi', 'Al Kut', 'Quchan', 'Saida2', 'Dieppe', 'Pinsk', 'Orsk', 'Kosti', 'Labinsk', 'Koszalin', 'Birnin Kebbi', 'Kaposvar', 'Glazov', 'Kovel', 'Adigrat', 'Stralsund', 'Touggourt', 'Cuamba', 'Tshikapa', 'Kpalime', 'Kadugli', 'Siedlce', 'Skelleftea', 'Birjand', 'Karaman', 'Dimitrovgrad', 'Tanga', 'Beziers', 'Tartu', 'Bida', 'Murom', 'Baia Mare', 'Balashov', 'Abha', 'Kovrov', 'Al Jubayl', 'Mascara', 'Durres', 'Kitale', 'Nizhnyaya Tura', 'Bolzano', 'Chipata', 'Sivas', 'Atyrau', 'Bandar-e Bushehr', 'Molepolole', 'Daugavpils', 'Orsha', 'Berberati', 'Nikopol', 'Ar Raqqah', 'Presov', 'Djelfa', 'Shendi', 'Khvoy', 'Calarasi', 'Guider', 'Kilosa', 'Lahij', 'Arezzo', 'Pec2', 'Rivne', 'Tabora', 'Almetyevsk', 'Leeuwarden', 'Szolnok', 'Kalmar', 'Ajaccio', 'Potchefstroom', 'Eregli', 'Batumi', 'Hof', 'Tirgu Mures', 'Nevers', 'Velikiye Luki', 'Lichinga', 'Balti', 'Pec1', 'Stavanger', 'Grudziadz', 'Mwene-Ditu', 'Qazvin', 'Severodvinsk', 'Trollhattan', 'Boras', 'Passau', 'Rustavi', 'Zielona Gora', 'Chalkida', 'Bastia', 'Pervouralsk', 'Izmayil', 'Bururi', 'Mtwara', 'Kupyansk', 'Rashid', 'Nabeul', 'Varna', 'El Fasher', 'Batna', 'Tiraspol', 'Gemena', 'Gonbad-e Kavus', 'Abadan', 'Novorossiysk', 'Vaxjo', 'Tuzla', 'Ostersund', 'Chingola', 'Zarzis', 'Medea', 'Brive', 'Kalamata', 'Osijek', 'Ioanina', 'Al Hufuf', 'Vaduz', 'Kirikkale', 'Gedaref', 'Serpukhov', 'Rosenheim', 'Nazareth', 'Esbjerg', 'Hafar al Batin', 'As Sulaymaniyah', 'Zlatoust', 'Biskra', 'Toamasina', 'Brindisi', 'Polokwane', 'Kremenchuk', 'Mutare', 'Nis', 'Jena', 'Klaipeda', 'Likasi', 'Starsy Oskol', 'Gweru', 'Gyor', 'Larissa', 'Dayr az Azwr', 'Lappeenranta', 'Chimoio', 'Karbala', 'Erzurum', 'Ternopil', 'Harar', 'Agen', 'Legnica', 'Mostar', 'Engels', 'El Oued', 'Mmabatho', 'Qena', 'Pardubice', 'Kuopio', 'Bafra', 'Barysaw', 'An Nasiriyah', 'Vichy', 'Al Kharj', 'Iraklio', 'Flensburg', 'As Samawah', 'Ardabil', 'Lutsk', 'Siracusa', 'Soke', 'Sumy', 'Qunghirot', 'Berezniki', 'Gera', 'Musoma', 'Lysychansk', 'Calais2', 'Ras al Khaymah', 'Umea', 'Bizerte', 'Lugano', 'Ivano-Frankivsk', 'Afyon', 'Giessen', 'Ruse', 'Maribor', 'Ancona', 'Hrodna', 'Armavir', 'Pizen', 'Kabwe', 'Oral', 'Halmstad', 'Klagenfurt', 'Zlin', 'Cherkessk', 'Erzincan', 'Gombe', 'Craiova', 'Vladimir', 'Al Amarah', 'Trento', 'Ferrara', 'Khoramabad', 'Plock', 'Gusau', 'Nizhyn', 'Kosice', 'Banja Luka', 'Entebbe', 'Kalisz', 'Trieste', 'Bacau', 'Dese', 'Kimberley', 'Minna', 'Kirovohrad', 'Bourges', 'Gottingen', 'Cottbus', 'Catanzaro', 'Pecs', 'Babruysk', 'Parma', 'Klerksdorp', 'Luzern', 'Ndola', 'Lemosos', 'Brasov', 'Livorno', 'Dasoguz', 'Ploiesti', 'Al Karak', 'Samandagi', 'Ipswich', 'Arak', 'Orebro', 'Bolu', 'Rybinsk', 'Abu Kamal', 'Irbil', 'Hail', 'Bremerhaven', 'Troyes', 'Helsingborg', 'Ravenna', 'Debrecen', 'Amiens', 'Le Mans', 'Drammen', 'Groningen', 'Bukavu', 'Kostroma', 'Tarragona', 'Kerman', 'Kherson', 'Taranto', 'Baqubah', 'Mahilyow', 'Lausanne', 'Balikesir', 'Cherepovets', 'Novi Sad', 'Cherkasy', 'Karlstad', 'Reims', 'Timisoara', 'Aalborg', 'Vinnytsya', 'Amol', 'Sari', 'Messina', 'Norwich', 'Iasi', 'Belgorod', 'Noginsk', 'Bauchi', 'Suhar', 'Lafia', 'Orel', 'Dezful', 'Szeged', 'Innsbruck', 'Saransk', 'Poltava', 'Vitsyebsk', 'Miskolc', 'Kaluga', 'Cheboksary', 'Split', 'Bouira', 'Gavle', 'Kumba', 'Van', 'Eskisehir', 'Lecce', 'Gaziantep', 'Bialystok', 'Qom', 'Gabes', 'Jyvaskyla', 'Manisa', 'Al Hudaydah', 'Zaria', 'Batman', 'Kermanshah', 'Zonguldak', 'Syktyvkar', 'Luhansk', 'Isiro', 'Buraydah', 'Norrkoping', 'Skikda', 'Sadah', 'Yoshkar Ola', 'Nakuru', 'Gorgan', 'Iskenderun', 'Freiburg', 'Wurzburg', 'Ulm', 'Kendu Bay', 'Uppsala', 'Melitopol', 'Adiyaman', 'Dijon', 'Nimes', 'Volzhskiy', 'Kilifi', 'Namur', 'Diyarbakir', 'Podolsk', 'Medani', 'Brugge', 'M  nster', 'Vladikavkaz', 'Malatya', 'Odense', 'Shakhty', 'Linkoping', 'Ordu', 'Simferopol', 'Ingolstadt', 'Salzburg', 'Ivanovo', 'Nowy Sacz', 'Perugia', 'Regensburg', 'Chernivtsi', 'Hamah', 'Lira', 'Al Hillah', 'Radom', 'Bukoba', 'Beni Suef', 'Nizhnekamsk', 'Limoges', 'Gdynia', 'Cagliari', 'Ad Damman', 'Lublin', 'Umtata', 'Rostock', 'Potsdam', 'Hims', 'Dzerzhinsk', 'Rasht', 'Denizli', 'Magdeburg', 'Kaunas', 'Bydgoszcz', 'Saint-Etienne', 'Szczecin', 'Kisumu', 'Kursk', 'Maxixe', 'Zaporizhzhya', 'Kocaeli', 'Nablus', 'Mariupol', 'Lubeck', 'Haarlem', 'Kiel', 'Nkhotakota', 'Tarsus', 'Mulhouse', 'Aarhus', 'Kielce', 'Douala', 'Moanda1', 'Palma', 'Zabid', 'Pescara', 'Ismailia', 'Lipetsk', 'Haifa', 'Osnabr  ck', 'Funtua', 'Nancy', 'Brno', 'Oulu', 'Gliwice', 'Pietermaritzburg', 'Genoa', 'Mangochi', 'Tula1', 'Izhevsk', 'Al Ayn', 'Ijebu Ode', 'Tukuyu', 'Sfax', 'Turku', 'Pisa', 'Montpellier', 'Orleans', 'Grenoble', 'Brighton', 'Tours', 'Benin City', 'Kayseri', 'Hatay', 'Graz', 'Vatican City', 'Rouen', 'Tolyatti', 'Nurnberg', 'Rzeszow', 'Bari', 'Toulon', 'Wroclaw', 'Antalya', 'Basel', 'Koblenz', 'Giyon', 'Khomeini Shahr', 'Linz', 'Leipzig', 'Beni Mazar', 'Beni', 'Bejaia', 'Bologna', 'Trablous', 'Utrecht', 'Salima', 'Mubi', 'Poznan', 'Charleroi', 'Abeokuta', 'Verona', 'Ilorin', 'Yola', 'Sapele', 'Hajjah', 'Aba', 'Wiesbaden', 'Ostrava', 'Hannover', 'Mbale', 'Lodz', 'Bonn', 'Goteborg', 'Mannheim', 'Trabzon', 'Mallawi', 'Metz', 'Chlef', 'Jos', 'Bremen', 'Mbarara', 'Salerno', 'Kabale', 'Katowice', 'Toulouse', 'Turin', 'Nice', 'Tartus', 'Karaj', 'Thohoyandou', 'Rotterdam', 'Saarbrucken', 'Koln', 'D  sseldorf', 'Arua', 'Sohag', 'Stuttgart', 'Moshi', 'Bergamo', 'El Giza', 'Antwerpen', 'Tororo', 'Abomey', 'Versailles', 'Zomba', 'Ikare', 'Dila', 'Jima', 'Masaka', 'Byumba', 'Tanta', 'Muyinga', 'Orlu', 'Katsina', 'Sodo', 'Hosaina', 'Hammerfest', 'Kasongo', 'Springbok', 'Hamadan', 'Middelburg1', 'L  deritz', 'Mahajanga', 'Berbera', 'Zuwarah', 'Bambari', 'Assab', 'Ebolowa', 'Venice', 'Namibe', 'Kayes1', 'Banguela', 'Al Jawf', 'Kalemie', 'Walvis Bay', 'Quelimane', 'Turkmenbasy', 'Lobamba', 'Malakal', 'Antsiranana', 'Juba', 'Andorra', 'Mekele', 'Atakpame', 'Xai-Xai', 'Upington', 'Bandundu', 'Arusha', 'Tete', 'Rovaniemi', 'Worcester2', 'Geneina', 'Mahalapye', 'Tahoua', 'Kristiansand', 'Franceville', 'Bur Said', 'Luanshya', 'Mufulira', 'Livingstone', 'Moundou', 'Morogoro', 'Parakou', 'Malanje', 'Sochi', 'Monaco', 'Lubango', 'Garoua', 'Hargeysa', 'Chitungwiza', 'Mbandaka', 'Dire Dawa', 'Serowe', 'Francistown', 'Kroonstad', 'Brest2', 'Magnitogorsk', 'Toliara', 'Constanta', 'Lulea', 'George', 'Bahir Dar', 'Petrozavodsk', 'Vaasa', 'Pemba', 'Cluj-Napoca', 'Aqtobe', 'El Obeid', 'Dodoma', 'Pskov', 'Bandar-e-Abbas', 'Zhytomyr', 'Aswan', 'Luxembourg', 'Maradi', 'Voronezh', 'Nukus', 'Misratah', 'Gulu', 'El Suweis', 'Vologda', 'Mzuzu', 'Stavropol', 'Gaza', 'Butembo', 'Tabuk', 'Smolensk', 'Pointe-Noire', 'Tver', 'Sundsvall', 'Sousse', 'Kirkuk', 'Yazd', 'Nampula', 'Eldoret', 'Ulyanovsk', 'Kirov2', 'Ogbomosho', 'Bloemfontein', 'Makurdi', 'Annaba', 'Constantine', 'Malmo', 'Naltchik', 'Fianarantsoa', 'Tabriz', 'Orenburg', 'An Najaf', 'Bryansk', 'Le Havre', 'Kaliningrad', 'Penza', 'Nizhny Tagil', 'Kisangani', 'Basra', 'Mosul', 'Astrakhan', 'Gdansk', 'Ryazan', 'Kananga', 'Krasnodar', 'Lvov', 'Maiduguri', 'East London', 'Yaroslave', 'Medina', 'Mbuji-Mayi', 'Adana', 'Saratov', 'Abuja', 'Odessa2', 'Geneva', 'Rostov1', 'Konya', 'The Hague', 'Ahwaz', 'Aleppo', 'Groznyy', 'El Faiyum', 'Mashhad', 'Tampere', 'Dnipropetrovsk', 'Catania', 'Bursa', 'Omdurman', 'Porto-Novo', 'Asyut', 'Marseille', 'Kaduna', 'Bamenda', 'Donetsk', 'Kharkiv', 'Mwanza', 'Ibadan', 'Dresden', 'Samsun', 'Florence3', 'Izmir', 'Jeddah', 'Tel Aviv-Yafo', 'Warri', 'Krakow', 'Port Harcourt', 'Goma', 'Hamburg', 'Meru', 'Blantyre', 'Taizz', 'Malabo', 'Dongola', 'Ndele', 'Obo', 'Matadi', 'Grootfontein', 'Abeche', 'Tamanrasset', 'Victoria4', 'Bumba', 'Port-Gentil', 'Menongue', 'Tromso', 'Ghardaia', 'Mbabane', 'Kikwit', 'Sabha', 'Wau', 'Agadez', 'Sao Tome', 'Aden', 'Gonder', 'Kolwezi', 'Podgorica', 'Pristina', 'Djibouti', 'Zinder', 'Huambo', 'Maroua', 'Mbeya', 'St.-Denis', 'Kassala', 'Moroni', 'Port Sudan', 'Port Louis', 'Gaborone', 'Tirana', 'Murmansk', 'Bern', 'Niyala', 'Ndjamena', 'Zanzibar', 'Trondheim', 'Beira', 'Windhoek', 'Asmara', 'Bangui', 'Bergen', 'Banghazi', 'Abu Dhabi', 'Mogadishu', 'Sarajevo', 'Muscat', 'Sevastapol', 'Valletta', 'Libreville', 'Ashgabat', 'Chisinau', 'Lubumbashi', 'Bratislava', 'Skopje', 'Nacala', 'Archangel', 'Niamey', 'Vilnius', 'Nicosia', 'Tallinn', 'Tbilisi', 'Maseru', 'Ljubljana', 'Brazzaville', 'Volgograd', 'Sanaa', 'Luxor', 'Thessaloniki', 'Riga', 'Manama', 'Lusaka', 'Samara', 'Maputo', 'Yerevan', 'Cotonou', 'Tripoli1', 'Yaounde', 'Port Elizabeth', 'Shiraz', 'Belgrade', 'Isfahan', 'Dar es Salaam', 'Minsk', 'Makkah', 'Kiev', 'Sofia', 'Ufa', 'Kazan', 'Luanda', 'Zagreb', 'Baku', 'Jerusalem', 'Bulawayo', 'Kuwait', 'Nizhny Novgorod', 'Doha', 'Palermo', 'Perm', 'Sokoto', 'Harare', 'Prague', 'Oslo', 'Mombasa', 'Athens2', 'Lome', 'Bucharest', 'Kinshasa', 'Khartoum', 'Alexandria3', 'Amman', 'Kampala', 'Beirut', 'Accra', 'Addis Ababa', 'Tunis', 'Tehran', 'Pretoria', 'St. Petersburg2', 'Helsinki', 'Ankara', 'Damascus', 'Cape Town', 'Durban', 'Budapest', 'Naples2', 'Baghdad', 'Kigali', 'Lyon', 'Nairobi', 'Antananarivo', 'Barcelona1', 'Warsaw', 'Riyadh', 'Algiers', 'Lagos', 'Bujumbura', 'Lilongwe', 'Enugu', 'Kano', 'Dubayy', 'Z  rich', 'Amsterdam', 'Copenhagen', 'Munich', 'Vienna', 'Rome', 'Frankfurt', 'Stockholm', 'Berlin', 'Brussels', 'Milan', 'Paris', 'Johannesburg', 'Istanbul', 'Moscow', 'Cairo', 'London2'])


In [ ]:
#l5=list(['Danjiangkou', 'Sharbaqty', 'Southern Cross', 'Sanming', 'Morawa', 'Sherlovaya Gora', 'Phayao', 'Kualakapuas', 'Thongwa', 'Wagin', 'Aqadyr', 'Chernogorsk', 'Mount Barker', 'Roebourne', 'Osakarovka', 'Si Racha', 'Tom Price', 'Severnyy', 'Makale', 'Uliastay', 'Bua Yai', 'Padangpanjang', 'Zixing', 'Kupina', 'Wuchuan', 'Aranyaprathet', 'Buyant-Uhaa', 'Zhosaly', 'Hon Quan', 'Shemonaikha', 'Abaza', 'Shangdu', 'Zaozernyy', 'Arvayheer', 'Chabahar', 'Tsetserleg', 'Shilka', 'Cholpon Ata', 'Pol-e Khomri', 'Erenhot', 'Yaynangyoung', 'Kyakhta', 'Bogotol', 'Kentau', 'Loei', 'Toguchin', 'Plast', 'Ust Kut', 'Rudny', 'Atbasar', 'Krasnouralsk', 'Parachinar', 'Karokh', 'Nizhenvartovsk', 'Tejen', 'Zuozhou', 'Wakema', 'Verkhniy Ufaley', 'Pakxe', 'Naryn', 'Narathiwat', 'Severouralsk', 'Tongren', 'Alapayevsk', 'Kompong Chhnang', 'Padangsidempuan', 'Petrovsk Zabaykalskiy', 'Tanjungkarang-Telubketung', 'Lahad Datu', 'Tak', 'Gangtok', 'Esik', 'Hong Gai', 'Hanggin Houqi', 'Lenger', 'Tayshet', 'Shwebo', 'Zharkent', 'Kuga', 'Kwinana', 'Perabumulih', 'Asbest', 'Iskitim', 'Kiselevsk', 'Nakhon Phanom', 'Teluk Intan', 'Pakokku', 'Surin', 'Tengchong', 'Qyzylorda', 'Kansk', 'Mawlamyine', 'Batu Pahat', 'Leninsk Kuznetsky', 'Keluang', 'Guliston', 'Iskandar', 'Sheberghan', 'Anzhero Sudzhensk', 'Sawahlunto', 'Kon Tum', 'Banyuwangi', 'Usolye Sibirskoye', 'Hospet', 'Tokmak', 'Sibu', 'Phan Rang', 'Taiping', 'Udon Thani', 'Kara Balta', 'Monywa', 'Pangkalpinang', 'Tebingtinggi', 'Panzhihua', 'Muar', 'Tan An', 'Chuxiong', 'Chirchiq', 'Lop Buri', 'Denpasar', 'Lhokseumawe', 'Pinrang', 'Urgut', 'Letpadan', 'Lampang', 'Samarinda', 'Anuradhapura', 'Parbhani', 'Kurnool', 'Sholapur', 'Dong Hoi', 'Mengzi', 'Seremban', 'Nakhon Si Thammarat', 'Karimnagar', 'Xuanhua', 'Shiyan', 'Martapura', 'Hat Yai', 'Konibodom', 'Nizamabad', 'Kohat', 'Hangu', 'Phan Thiet', 'Khujand', 'Ajmer', 'Mataram', 'Mirput Khas', 'Raurkela', 'Denow', 'Kota Kinabalu', 'Chirala', 'Hetauda', 'Qurghonteppa', 'Kattaqorgon', 'Shihezi', 'Bhiwandi', 'Linxia', 'Johor Bahru', 'Tirunelveli', 'Ca Mau', 'Zhuzhou', 'Lalitpur', 'Sopur', 'Yangquan', 'Nonthaburi', 'Tiruppur', 'Pathum Thani', 'Bhusawal', 'Badulla', 'Hisar', 'Namangan', 'Zhucheng', 'Kalyan', 'Yichang', 'Butterworth', 'Zhenjiang', 'Shashi', 'Abbottabad', 'Soc Trang', 'Moratuwa', 'Samut Prakan', 'Son Tay', 'Long Xuyen', 'Sangli', 'Xinyu', 'Mansehra', 'Pasuruan', 'Galle', 'Pakalongan', 'Bien Hoa', 'Hoshiarpur', 'Anlu', 'Chau Doc', 'Yangzhou', 'Rahimyar Khan', 'Hapur', 'Huizhou', 'Magelang', 'Guntur', 'Dali1', 'Ghaziabad', 'Narayanganj', 'Karur', 'Hathras', 'Cuddalore', 'Bac Giang', 'Foshan', 'Jorhat', 'Mathura', 'Chaozhou', 'Anyang', 'Sonipat', 'Jiaozuo', 'Shenzhen', 'Cirebon', 'Changzhou', 'Mojokerto', 'Tianmen', 'Rohtak', 'Chiniot', 'Ambala', 'Hengshui', 'Vadodara', 'Haora', 'Taian', 'Bhubaneshwar', 'Firozabad', 'Thanjavur', 'Bhiwani', 'Vellore', 'Sialkote', 'Pilibhit', 'Mardan', 'Muzaffarnagar', 'Kumbakonam', 'Bekasi', 'Madiun', 'Sheikhu Pura', 'Bulandshahr', 'Gurgaon', 'Gujrat', 'Moradabad', 'Budaun', 'Faisalabad', 'Xinyi', 'Agartala', 'Luohe', 'Rajahmundry', 'Kaifeng', 'Sylhet', 'Tangail', 'Deyang', 'Zigong', 'Sitapur', 'Sahiwal', 'Barddhaman', 'Allahabad', 'Amderma', 'Novyy Urengoy', 'Vikhorevka', 'Bamian', 'Linxi', 'Saryshaghan', 'Shaowu', 'Shira', 'Three Springs', 'Labutta', 'Meekatharra', 'Navsari', 'Newman', 'Tarutung', 'Vizianagaram', 'Kirensk', 'Huize', 'Exmouth', 'Gwadar', 'Zhaltyr', 'Kargat', 'Mandalgovi', 'Uray', 'Xiangkhoang', 'Zholymbet', 'Suntar', 'Tobol', 'Katanning', 'Namtu', 'Kachiry', 'Merredin', 'Thung Song', 'Bulaevo', 'Olovyannaya', 'Qarqaraly', 'Nadym', 'Arxan', 'Komsomolets', 'Alxa Zuoqi', 'Gingin', 'Lijiang', 'Qusmuryn', 'Esil', 'Shonzhy', 'Guide', 'Narrogin', 'Kyaukphyu', 'Shar', 'Manjimup', 'Tayynsha', 'Zmeinogorsk', 'Artemovsk', 'Tash Komur', 'Uthai Thani', 'Dudinka', 'Sungaipenuh', 'Quiemo', 'Gornyak', 'Cam Ranh', 'Cherlak', 'Pursat', 'Ulaangom', 'Feyzabad', 'Khilok', 'Megion', 'Northam', 'Qapshaghay', 'Ertis', 'Sayanogorsk', 'Kulunda', 'Krong Koh Kong', 'Derzhavinsk', 'Qulan', 'Timiryazevskiy', 'Igarka', 'Gusinoozyorsk', 'Zarafshon', 'Anxi', 'Zheleznogorsk Ilimskiy', 'Ha Giang', 'Qarazhal', 'Ivdel', 'Takeo', 'Erymentau', 'Ondorhaan', 'Yeniseysk', 'Khorugh', 'Dzuunharaa', 'Pingxiang1', 'Wuyuan', 'Moron2', 'Zaysan', 'Qaratau', 'Golyshmanovo', 'Krasnokamensk', 'Asino', 'Phyarpon', 'Polevskoy', 'Pyu', 'Kuybyshevskiy', 'Salekhard', 'Bayan Obo', 'Urzhar', 'Barabinsk', 'Lesosibirsk', 'Zhob', 'Kurtamysh', 'Slyudyanka', 'Strezhevoy', 'Yandoon', 'Kamenna Obi', 'Ushtobe', 'Cherepanovo', 'Hua Hin', 'Nazyvayevsk', 'Zhetiqara', 'Biryusinsk', 'Nyagan', 'Sukhothai', 'Myeik', 'Aral', 'Atasu', 'Karasuk', 'Makinsk', 'Topki', 'Borzya', 'Mogocha', 'Olgiy', 'Yurga', 'Inta', 'Khammouan', 'Lahat', 'Slavgorod', 'Hoa Binh', 'Kalachinsk', 'Balqash', 'Allanmyo', 'Sisophon', 'Yalutorovsk', 'Aleysk', 'Pathankot', 'Ayakoz', 'Shieli', 'Nevyansk', 'Uzhur', 'Lao Chi', 'Artemovskiy', 'Nerchinsk', 'Tatarsk', 'Uyar', 'Karpinsk', 'At Bashy', 'Torbat-e Jam', 'Sarqan', 'Raub', 'Pyay', 'Manzhouli', 'Atamyrat', 'Otar', 'Isikul', 'Tara', 'Ob', 'Singkawang', 'Arqalyq', 'Ye', 'Tashtagol', 'Arys', 'Bunbury', 'Toktogul', 'Gilgit', 'Gorno Altaysk', 'Karamay', 'Kartaly', 'Tanjungpinang', 'Nizhneudinsk', 'Prachin Buri', 'Verkhnyaya Salda', 'Cam Pha', 'Khanty Mansiysk', 'Guryevsk', 'Kokshetau', 'Kyshtym', 'Pathein', 'Chauk', 'Sisaket', 'Tacheng', 'Balykchy', 'Ridder', 'Sampit', 'Yemanzhelinsk', 'Ekibastuz', 'Zima', 'Zyryanovsk', 'Changting', 'Tobolsk', 'Saraburi', 'Shadrinsk', 'Mariinsk', 'Buriram', 'Songkhla', 'Xilinhot', 'Dawei', 'Gardiz', 'My Tho', 'Yishan', 'Fuan', 'Nanping', 'Port Blair', 'Minxian', 'Lishui', 'Kampot', 'Jinchang', 'Rayong', 'Chenzhou', 'Valparai', 'Shu', 'Savannakhet', 'Louangphrabang', 'Sakhon Nakhon', 'Tavda', 'Yala', 'Tulun', 'Quang Tri', 'Talas', 'Mae Sot', 'Simao', 'Korla', 'Farah', 'Chumphon', 'Turkistan', 'Cheremkhovo', 'Chaiyaphum', 'Turpan', 'Puerto Princesa', 'Kampong Cham', 'Troitsk', 'Temirtau', 'Oytal', 'Magway', 'Turbat', 'Chukai', 'Ishim', 'Rubtsovsk', 'Aizawl', 'Shaoyang', 'Chanthaburi', 'Novoaltaysk', 'Kulob', 'Andkhvoy', 'Taungoo', 'Wenshan', 'Ratchaburi', 'Taldyqorghan', 'Olmaliq', 'Pervouralsk', 'Karakol', 'Myingyan', 'Bintulu', 'Serov', 'Kuantan', 'Trang', 'Zahedan', 'Nakhon Sawan', 'Hinthada', 'Petropavlovsk', 'Chiang Rai', 'Bijapur', 'Achinsk', 'Palangkaraya', 'Vung Tau', 'Oostanay', 'Play Ku', 'Myitkyina', 'Phetchaburi', 'Uttaradit', 'Bago', 'Aksu', 'Batticaloa', 'Sandakan', 'Tonk', 'Tawau', 'Fengzhen', 'Phuket', 'Pingliang', 'Kogon', 'Nan', 'Miass', 'Yunxian', 'Dibrugarh', 'Ayutthaya', 'Dasoguz', 'Mudon', 'Hindupur', 'Nandyal', 'Tuticorin', 'Bengkulu', 'Pematangsiantar', 'Nha Trang', 'Linchuan', 'Miri', 'Zabol', 'Fengjie', 'Angren', 'Urgentch', 'Jining1', 'Kolar', 'Uroteppa', 'Da Lat', 'Biysk', 'Lianxian', 'Ninde', 'Buxoro', 'Davangere', 'Longyan', 'Nong Khai', 'Ha Tinh', 'Salatiga', 'Kuta', 'Ghazni', 'Prokopyevsk', 'Phitsanulok', 'Phrae', 'Bikaner', 'Hunanghua', 'Raichur', 'Ongole', 'Pali', 'Kamensk Uralskiy', 'Chengde', 'Proddatur', 'Dali2', 'Termiz', 'Porbandar', 'Burhanpur', 'Puqi', 'Ubon Ratchathani', 'Zhangjiakou', 'Singaraja', 'Xingyi', 'Navoi', 'Tongchuan', 'Zhijiang', 'Bhilwara', 'Sagar', 'Bidar', 'Tumkur', 'Jingdezhen', 'Khiwa', 'Chon Buri', 'Hancheng', 'Baghlan', 'Jizzax', 'Latur', 'Meizhou', 'Ahmednagar', 'Mandya', 'Xiamen', 'Sibolga', 'Jian', 'Xichang', 'Anqing', 'Kuala Terengganu', 'Kurgan', 'Sambalpur', 'Pakanbaru', 'Shouzhou', 'Pontianak', 'Xinzhou', 'Ratlam', 'Jalal Abad', 'Jambi', 'Rajkot', 'Amravati', 'Bhavnagar', 'Dhule', 'Khammam', 'Buon Me Thuot', 'Ipoh', 'Yuxi', 'Meymaneh', 'Khon Kaen', 'Osh', 'Udaipur', 'Shishou', 'Heyuan', 'Tyumen', 'Bellary', 'Hubli', 'Tongling', 'Jiujiang', 'Ujjain', 'Sungai Petani', 'Qinzhou', 'Qarshi', 'Kondoz', 'Jaffna', 'Rajapalaiyam', 'Chandrapur', 'Maanshan', 'Liuzhou', 'Vinh', 'Shillong', 'Longxi', 'Baramula', 'Haikou', 'Alor Setar', 'Guangyuan', 'Macau', 'Zicheng', 'Kota Baharu', 'Shimoga', 'Beihai', 'Belgaum', 'Majene', 'Datong', 'Kilinochchi', 'Balkh', 'Gwalior', 'Qingyuan', 'Malacca', 'Omsk', 'Matara', 'Langfang', 'Rach Gia', 'Qui Nhon', 'Viet Tri', 'Zhanyi', 'Yongzhou', 'Shahrisabz', 'Aurangabad2', 'Zhaoqing', 'Zhaotang', 'Jalalabad', 'Bilaspur', 'Dera Ismail Khan', 'Malegaon', 'Palu', 'Quang Ngai', 'Changping', 'Guangshui', 'Ganzhou', 'Yangjiang', 'Thai Nguyen', 'Bac Lieu', 'Binjai', 'Jhansi', 'Jamshedpur', 'Dehra Dun', 'Bannu', 'Kelang', 'Zhangzhou', 'Changde', 'Jincheng', 'Shangrao', 'Fargona', 'Hue', 'Yuci', 'Can Tho', 'Sirsa', 'Wuhu', 'Imphal', 'Jinshi', 'Mirzapur', 'Puri', 'Nagercoil', 'Nellore', 'Quzhou', 'Bukittinggi', 'Rizhao', 'Anshun', 'Warangal', 'Sikar', 'Xiangtan', 'Hefei', 'Tirupati', 'Wanxian', 'Qoqon', 'Putian', 'Sadiqabad', 'Jingmen', 'Tasikmalaya', 'Charikar', 'Nawabshah', 'Luan', 'Abohar', 'Changzhi', 'Machilipatnam', 'Xinyang', 'Hebi', 'Kundian', 'Fengcheng', 'Linfen', 'Saidu', 'Nasik', 'Pingyi', 'Yaan', 'Dindigul', 'Pingdu', 'Kakinada', 'Probolinggo', 'Cangzhou', 'Bogor', 'Alwar', 'Qinhuangdao', 'Brahmapur', 'Hanzhong', 'Nepalganj', 'Zunyi', 'Praya', 'Sukabumi', 'Kediri', 'Vijayawada', 'Laiwu', 'Cilacap', 'Dera Ghazi Khan', 'Jiexiu', 'Ninh Binh', 'Nangong', 'Bharatpur', 'Huzhou', 'Yibin', 'Jember', 'Buizhou', 'Silchar', 'Sumenep', 'Agra', 'Blitar', 'Tegal', 'Xinxiang', 'Thanh Hoa', 'Serang', 'Yiyang', 'Bengbu', 'Xiantao', 'Gujranwala', 'Chandigarh', 'Bareilly', 'Wuxi', 'Yishui', 'Aurangabad1', 'Panipat', 'Quilon', 'Rampur', 'Baoji', 'Allappey', 'Xiaogan', 'Gandhinagar', 'Weinan', 'Yogyakarta', 'Sukkur', 'Indramayu', 'Tiruvannamalai', 'Tezpur', 'Salem3', 'Jullundur', 'Maoming', 'Meerut', 'Cochin', 'Jiangmen', 'Dezhou', 'Karnal', 'Xiangtai', 'Haldia', 'Jhang', 'Jammu', 'Kasur', 'Shuyang', 'Bahawalpur', 'Sargodha', 'Pati', 'Baleshwar', 'Hechi', 'Nam Dinh', 'Weifang', 'Kanchipuram', 'Aligarh', 'Tuban', 'Zaozhuang', 'Tangshan', 'Bhatpara', 'Rawalpindi', 'Linqing', 'Jamalpur', 'Patiala', 'Huangshi', 'Okara', 'Etawah', 'Dengzhou', 'Baoding', 'Pabna', 'Xuzhou', 'Zibo', 'Dingzhou', 'Xuchang', 'Jessore', 'Kotabumi', 'Leshan', 'Barisal', 'Bahraich', 'Pingdingshan', 'Fatehpur', 'Gaya2', 'Krishnanagar', 'Shahjahanpur', 'Nanyang', 'Medinipur', 'Liaocheng', 'Heze', 'Comilla', 'Shanxian', 'Puyang', 'Kozhikode', 'Siliguri', 'Alipur Duar', 'Jining2', 'Biratnagar', 'Luoyang', 'Langzhong', 'Lianyungang', 'Luzhou', 'Rangpur', 'Huaibei', 'Mymensingh', 'Taizhou', 'Fuyang', 'Saidpur', 'Faizabad', 'Zhoukou', 'Dhanbad', 'Shangqiu', 'Purnia', 'Jieshou', 'Neijiang', 'Birganj', 'Nawabganj', 'Nanchong', 'Muzaffarpur', 'Gorakhpur', 'Fulin', 'Bukachacha', 'Qazaly', 'Port Hedland', 'Busselton', 'Karratha', 'Dalandzadgad', 'Mirnyy', 'Altay', 'Dund-Us', 'Ust-Ulimsk', 'Shizuishan', 'Xigaze', 'Lensk', 'Mandurah', 'Albany4', 'Tanjungpandan', 'Sittwe', 'Gejiu', 'Geraldton1', 'Choybalsan', 'Vorkuta', 'Siem Reap', 'Shache', 'Parepare', 'Yulin2', 'Golmud', 'Dunhuang', 'Abakan', 'Kyzyl', 'Sanya', 'Hailar', 'Battambang', 'Surat Thani', 'Bose', 'Balikpapan', 'Oskemen', 'Norilsk', 'Mazar-e Sharif', 'Pavlodar', 'Turkmenabat', 'Mary', 'Angarsk', 'Kuching', 'Semey', 'Yining', 'Raba', 'Taraz', 'Wuzhou', 'Ulan Ude', 'Bandar Lampung', 'Gulbarga', 'George Town2', 'Kemerovo', 'Nakhon Ratchasima', 'Shymkent', 'Quetta', 'Nizhny Tagil', 'Mandalay', 'Novokuznetsk', 'Baoshan', 'Herat', 'Zhanjiang', 'Banda Aceh', 'Akola', 'Kota', 'Chiang Mai', 'Shaoguan', 'Chifeng', 'Mangalore', 'Nanded', 'Hohhot', 'Jabalpur', 'Tianshui', 'Mysore', 'Chelyabinsk', 'Lupanshui', 'Da Nang', 'Wuwei', 'Guwahati', 'Jodhpur', 'Kolhapur', 'Putrajaya', 'Houma2', 'Ankang', 'Andijon', 'Jinhua', 'Yueyang', 'Raipur', 'Indore', 'Hengyang', 'Ranchi', 'Yulin1', 'Trivandrum', 'Macheng', 'Pondicherry', 'Qingdao', 'Malang', 'Kotte', 'Haiphong', 'Xiangfan', 'Quanzhou', 'Tiruchchirappalli', 'Khulna', 'Kandy', 'Srinagar', 'Saharanpur', 'Larkana', 'Linyi', 'Surakarta', 'Cuttack', 'Handan', 'Shijianzhuang', 'Rajshahi', 'Jinan', 'Ludhiana', 'Mianyang', 'Lucknow', 'Huaiyin', 'Chengdu', 'Bhagalpur', 'Male', 'Bratsk', 'Jiayuguan', 'Yinchuan', 'Tarakan', 'Surgut', 'Thimphu', 'Hami', 'Lhasa', 'Krasnoyarsk', 'Chita', 'Baotou', 'Tomsk', 'Qaraghandy', 'Naypyidaw', 'Astana', 'Hotan', 'Bandjarmasin', 'Barnaul', 'Irkutsk', 'Ulaanbaatar', 'Bandar Seri Begawan', 'Pingxiang2', 'Padang', 'Vientiane', 'Zhangye', 'Novosibirsk', 'Kashi', 'Xining', 'Almaty', 'Kathmandu', 'Fuzhou', 'Lanzhou', 'Bishkek', 'Yekaterinburg', 'Palembang', 'Kandahar', 'Nanning', 'Bhopal', 'Guiyang', 'Rangoon', 'Samarqand', 'Urumqi', 'Guilin', 'Kunming', 'Dushanbe', 'Coimbatore', 'Phnom Penh', 'Nagpur', 'Kabul', 'Nanjing', 'Tashkent', 'Taiyuan', 'Delhi', 'Karachi', 'Islamabad', 'Perth2', 'Hyderabad2', 'Surabaya', 'Ho Chi Minh City', 'Changsha', 'Medan', 'Pune', 'Hangzhou', 'Colombo', 'Hyderabad1', 'Visakhapatnam', 'Nanchang', 'Semarang', 'Peshawar', 'Tianjin', 'Surat', 'Huainan', 'Ujungpandang', 'Bandung', 'Jaipur', 'Chennai', 'Chittagong', 'Ahmadabad', 'Lahore', 'Wuhan', 'Bangalore', 'Calcutta', 'Madurai', 'Hanoi', 'Guangzhou', 'Zhengzhou', 'Amritsar', 'Dhaka', 'Shantou', 'Chongqing', 'Multan', 'Kanpur', 'Patna', 'Xian', 'Varanasi', 'Hong Kong', 'Singapore', 'Mumbai', 'Kuala Lumpur', 'Bangkok', 'New Delhi', 'Jakarta', 'Beijing', 'Shanghai'])

In [ ]:
#l6= list( ['Meningie', 'Queenstown2', 'Waitakere', 'Westport', 'Neryungri', 'Hoskins', 'Scottsdale1', 'Winton', 'Cloncurry', 'Oatlands', 'Barcaldine', 'Eumseong', 'Vilyuysk', 'Byron Bay', 'Kimhyonggwon', 'Mendi', 'Singleton', 'Chosan', 'Stawell', 'Anbyon', 'Colac', 'Leeton', 'Forbes', 'Ulladulla', 'Kavieng', 'Tumut', 'Atherton', 'Changyon', 'Grafton', 'Kaiapoi', 'Forster-Tuncurry', 'Hamilton4', 'Bugt', 'Ende', 'Maryborough', 'Ararat', 'Dalby', 'Parkes', 'Hervey Bay', 'Levin', 'Timika', 'Sunbury', 'Narrabri', 'Muswellbrook', 'Bongaree', 'Luwuk', 'Echuca', 'Uglegorsk', 'Kholmsk', 'Dalnerechensk', 'Xinqing', 'Kempsey', 'Kingston3', 'Tonghua', 'Gympie', 'Juifang', 'Angangxi', 'Timaru', 'Maitland', 'Traralgon', 'Burnie', 'Goulburn', 'Orongen Zizhiqi', 'Murray Bridge', 'Daan', 'Arsenyev', 'Lautoka', 'Shepparton', 'Warrnambool', 'Kanoya', 'Baiquan', 'Ergun Zuoqi', 'Linkou', 'Queanbeyan', 'Changling', 'Ozamis', 'Vanino', 'Lismore', 'Upper Hutt', 'Melton', 'Svobodnyy', 'Tweed Heads', 'Munchon', 'Zalantun', 'Tacloban', 'Capitol Hill', 'Rotorua', 'Tidore', 'Gyeongju', 'Suileng', 'Butuan', 'Caloundra', 'Manpo', 'Qingan', 'Partizansk', 'San Pablo', 'Qinggang', 'Caboolture', 'Nakhodka', 'Wonju', 'Chuncheon', 'Beipiao', 'Huangyan', 'Hamhung', 'Linhai', 'Takaoka', 'Anda', 'Cagayan de Oro', 'Iksan', 'Shimonoseki', 'Geelong', 'Hulan Ergi', 'Takapuna', 'Koriyama', 'Wafangdian', 'Yanji', 'Takamatsu', 'Komsomolsk na Amure', 'Hegang', 'Toyama', 'Otsu', 'Liaoyuan', 'Masan', 'Kaesong', 'Olongapo', 'Sunchon', 'Qitaihe', 'Tokushima', 'Cheongju', 'Pingtung', 'Fengshan', 'Naga', 'Utsunomiya', 'Okayama', 'Cranbourne', 'Bacolod', 'Ansan', 'Changhua', 'Liaoyang', 'Jilin', 'Kawasaki', 'Hsinchu', 'Maebashi', 'Chungli', 'Yuanlin', 'Hachioji', 'Kawagoe', 'Changzhou', 'Ashburton2', 'Bicheno', 'Chegdomyn', 'Kingston South East', 'Oamaru', 'Progress', 'Ravensthorpe', 'Streaky Bay', 'Abau', 'Gingoog', 'Hughenden', 'Richmond5', 'Theodore', 'Wewak', 'Batemans Bay', 'Cowell', 'Merimbula', 'Norseman', 'Ouyen', 'Wilcannia', 'Georgetown2', 'Port Douglas', 'Queenstown3', 'Leonara', 'Taedong', 'Yeppoon', 'Yulara', 'Kieta', 'Ceduna', 'Cobram', 'Hokitika', 'Penola', 'Amahai', 'Bordertown', 'Sohano', 'Yamba', 'Klyuchi', 'Berri', 'Central Coast', 'Makarov', 'Peterborough2', 'Wallaroo', 'Alotau', 'Pokrovsk', 'Longreach', 'Tennant Creek', 'Clare', 'Biloela', 'Palopo', 'Smithton', 'Charleville', 'Deniliquin', 'Goondiwindi', 'Proserpine', 'Seymour', 'Ingham', 'Tynda', 'Kununurra', 'Nabire', 'Kailu', 'Krasnogorsk', 'Tagum', 'Cooma', 'Roma', 'Savetskaya Gavan', 'Baubau', 'Moranbah', 'Wonthaggi', 'Inverell', 'Luganville', 'Innisfail', 'Dolinsk', 'Waingapu', 'Young', 'Musan', 'Popondetta', 'Port Augusta', 'Whakatane', 'Bathurst1', 'Bathurst2', 'Cowra', 'Ayr2', 'Charters Towers', 'Gunnedah', 'Aleksandrovsk', 'Sale', 'Sorong', 'Skovorodino', 'Victor Harbor', 'Ashburton1', 'Poronaysk', 'Swan Hill', 'Nandi', 'Cadiz1', 'Cadiz2', 'Maizaru', 'Mudgee', 'Horsham', 'Nevelsk', 'Nikolayevsk na Amure', 'Amursk', 'Vyazemskiy', 'Richmond4', 'Magdagachi', 'Poso', 'Kavalerovo', 'Sonbong', 'Esperance', 'Huinan', 'Port Pirie', 'Calbayog', 'Lithgow', 'Manokwari', 'Taree', 'Moree', 'Taupo', 'Yitulihe', 'Bairnsdale', 'Invercargill', 'Gawler', 'Bikin', 'Masterton', 'Emerald', 'Yian', 'Gannan', 'Tailai', 'Merauke', 'Kingaroy', 'Ballina', 'Warwick', 'Watampone', 'Mount Gambier', 'Pyongsan', 'Roxas', 'Shimanovsk', 'Ruteng', 'Mildura', 'Mary borough', 'Wangqing', 'Surigao', 'Madang', 'Linjiang', 'Coffs Harbour', 'Korsakov', 'Longjiang', 'Yichun', 'Nancha', 'Nehe', 'Gisborne', 'Ningan', 'Yakeshi', 'Gladstone', 'Sokcho', 'Katoomba', 'Boli', 'Nelson1', 'Andong', 'Liuhe', 'Ongjin', 'Tsuruoka', 'Biak', 'Nenjiang', 'Huanren', 'Shangzhi', 'Fujin', 'Tamworth', 'Nowra', 'Whangarei', 'Port Macquarie', 'Keshan', 'Kilchu', 'Taonan', 'New Plymouth', 'Mishan', 'Helong', 'Albury', 'Wanganui', 'Nampo', 'Belogorsk', 'Jiaohe', 'Hulin', 'Lesozavodsk', 'Hailun', 'Palmerston North', 'Shulan', 'Bundaberg', 'Spassk Dalniy', 'Lanxi', 'Otaru', 'Mackay', 'Noumea', 'Gangneung', 'Hyeson', 'Panshi', 'Haeju', 'Tieli', 'Mokpo', 'Tottori', 'Dunhua', 'Wagga Wagga', 'Kitami', 'Tomakomai', 'Jeju', 'Hungnam', 'Saseba', 'Sunshine Coast', 'Ussuriysk', 'Matsue', 'Birobidzhan', 'Chongju', 'Agana', 'Kanggye', 'Hamilton3', 'Kure', 'Pagadian', 'Manado', 'Shuangcheng', 'Taitung', 'Ballarat', 'Hachinohe', 'Kendari', 'Cotabato', 'Launceston', 'Fukui', 'Toowoomba', 'Hirosaki', 'Iligan', 'Zhaodong', 'Obihiro', 'Gunsan', 'Jiutai', 'Muroran', 'Ormac', 'Chongjin', 'Maumere', 'Hualien', 'Batangas', 'Nagaoka', 'Matsumoto', 'Aomori', 'Suihua', 'Wollongong', 'Iwaki', 'Akita', 'Legazpi', 'Fukushima', 'Yeosu', 'Kagoshima', 'Baicheng', 'Morioka', 'Yamagata', 'Jeonju', 'Fuyu', 'Nongan', 'Asahikawa', 'Pohang', 'Oita', 'Dandong', 'Benxi', 'Mito', 'Jixi', 'Shizuoka', 'Wakayama', 'Tieling', 'Kanazawa', 'Tungkang', 'Matsuyama', 'Zhuanghe', 'Niigata', 'Jinxi', 'Kofu', 'Ulsan', 'Fuxin', 'Jinzhou', 'Chaoyang', 'Kumamoto', 'Fushun', 'Naha', 'Laiyang', 'Tsu', 'Daqing', 'Daegu', 'Incheon', 'Tainan', 'Kitakyushu', 'Jiaojing', 'Hamamatsu', 'Gifu', 'Suwon', 'Yokohama', 'Yantai', 'Kobe', 'Taichung', 'Chiai', 'Yingkow', 'Pingdu', 'Shaoxing', 'Tarlac', 'Angeles', 'Huzhou', 'Suzhou', 'Wuxi', 'Dagupan', 'Cabanatuan', 'Jiaxing', 'Nantong', 'Lianyungang', 'Taizhou', 'Heihe', 'Broome', 'Rabaul', 'Dalnegorsk', 'Port Lincoln', 'Griffith', 'Katherine', 'Portland3', 'Jayapura', 'Aldan', 'Mount Isa', 'Wangaratta', 'Tumen', 'Devonport', 'Blenheim', 'Goroka', 'Okha', 'Armidale', 'Hastings', 'Sinuiju', 'Whyalla', 'Ternate', 'Kalgoorlie', 'Sakata', 'Broken Hill', 'Ambon', 'Orange', 'Tauranga', 'Mt. Hagen', 'Dubbo', 'Rockhampton', 'Wonsan', 'Kimchaek', 'Yuzhno Sakhalinsk', 'Blagoveshchensk', 'Dunedin', 'Bendigo', 'Beian', 'Lae', 'Cairns', 'Kushiro', 'Ulanhot', 'Hakodate', 'Keelung', 'Tongliao', 'Kochi', 'Miyazaki', 'Kupang', 'Zamboanga', 'Laoag', 'Manukau', 'Mudangiang', 'Gorontalo', 'Jiamusi', 'Siping', 'Weihai', 'Khabarovsk', 'Gold Coast', 'Daejeon', 'Nagano', 'Gwangju', 'Sendai', 'Kyoto', 'General Santos', 'Dalian', 'Anshan', 'Busan', 'Fukuoka', 'Pasay City', 'Ningbo', 'Quezon City', 'Wenzhou', 'Qingdao', 'Nagoya', 'Huaiyin', 'Palikir', 'Majuro', 'Melekeok', 'Port-Vila', 'Tarawa', 'Honiara', 'Magadan', 'Alice Springs', 'Dili', 'Petropavlovsk Kamchatskiy', 'Suva', 'Hobart', 'Darwin', 'Wellington', 'Yakutsk', 'Baguio City', 'Port Moresby', 'Nagasaki', 'Vladivostok', 'Townsville', 'Qiqihar', 'Kaohsiung', 'Christchurch', 'Auckland', 'Newcastle', 'Canberra', 'Hiroshima', 'Cebu', 'Davao', 'Sapporo', 'Pyongyang', 'Adelaide', 'Brisbane', 'Changchun', 'Shenyeng', 'Iloilo', 'Harbin', 'Hangzhou', 'Melbourne2', 'Manila', 'Taipei', 'Seoul', 'Sydney1', 'Osaka', 'Tokyo', 'Shanghai'] )

In [43]:
import pandas as pd
l1df = pd.DataFrame(l1, columns=['city'])
l2df = pd.DataFrame(l2, columns=['city' ])
l3df = pd.DataFrame(l3, columns=['city' ])
l4df = pd.DataFrame(l4, columns=['city' ])
l5df = pd.DataFrame(l5, columns=['city' ])
l6df = pd.DataFrame(l6, columns=['city' ])

l1df['SurfaceFile'] = '1'
l2df['SurfaceFile'] = '2'
l3df['SurfaceFile'] = '3'
l4df['SurfaceFile'] = '4'
l5df['SurfaceFile'] = '5'
l6df['SurfaceFile'] = '6'

#merged_df = pd.merge(l1df, l2df, on=['city'] )

#print(merged_df)
merged_df =  pd.concat([l1df, l2df, l3df, l4df, l5df, l6df], ignore_index=True).sort_values(by='city')
merged_df.to_csv('City_surface.csv', index=False, encoding="utf-8")
print(merged_df)

#import pandas as pd
# Reading a CSV file
#df = pd.read_csv('City_surface.csv')

merged_df["surface_file_name"] = merged_df.apply(lambda row: "SurfaceWater_2018_30m_30_"+str(pd.to_numeric(row.SurfaceFile)-1)+".tif", axis=1)



#for index, rows in df.iterrows():
print(merged_df)
merged_df.to_csv('City_surface.csv', encoding="utf-8")


            city SurfaceFile
3774     Aalborg           4
3893      Aarhus           4
3960         Aba           4
3612      Abadan           4
2032  Abaetetuba           3
...          ...         ...
2514  Zvishavane           4
3357      Zvolen           4
2034      Zwedru           3
4797  Zyryanovsk           5
4330     Z��rich           4

[6046 rows x 2 columns]
            city SurfaceFile               surface_file_name
3774     Aalborg           4  SurfaceWater_2018_30m_30_3.tif
3893      Aarhus           4  SurfaceWater_2018_30m_30_3.tif
3960         Aba           4  SurfaceWater_2018_30m_30_3.tif
3612      Abadan           4  SurfaceWater_2018_30m_30_3.tif
2032  Abaetetuba           3  SurfaceWater_2018_30m_30_2.tif
...          ...         ...                             ...
2514  Zvishavane           4  SurfaceWater_2018_30m_30_3.tif
3357      Zvolen           4  SurfaceWater_2018_30m_30_3.tif
2034      Zwedru           3  SurfaceWater_2018_30m_30_2.tif
4797  Zyryanovsk 

In [ ]:
fl1 = [item for item in l1 if item in l2]
print(fl1)

[]


In [ ]:
fl1 = [item for item in l2 if item in l3]
print(fl1)

['Chivilcoy', 'Manaus']


In [ ]:
fl1 = [item for item in l3 if item in l4]
print(fl1)

['Tarbes', 'Tema', 'Yendi', 'Brighton', 'Accra', 'London2']


In [ ]:
fl1 = [item for item in l4 if item in l5]
print(fl1)

['Severouralsk', 'Karpinsk', 'Pervouralsk', 'Dasoguz', 'Nizhny Tagil']


In [ ]:
fl1 = [item for item in l5 if item in l6]
print(fl1)

['Changzhou', 'Pingdu', 'Huzhou', 'Wuxi', 'Lianyungang', 'Taizhou', 'Qingdao', 'Huaiyin', 'Hangzhou', 'Shanghai']


In [ ]:
#filtlist = list()
#for l in len(citylist)-1:
#  #print(citylist[i].sort())
#  filtered_list1 = [item for item in citylist[i] if item not in citylist[i+1]]
#  filtlist.append(filtered_list1)
#  print(filtlist)

In [ ]:
# Surface Water Annual FY 2018  # Not used
import ee
image2 = ee.Image('JRC/GSW1_4/YearlyHistory/2018')   #.select(['Daytime', 'Nighttime'])

geemap.download_ee_image(image2, "SurfaceWater_2018_30_30.tif", region = worldGeometry, scale=30)


SurfaceWater_2018_30_30.tif: |          | 0.00/668k (raw) [  0.0%] in 00:00 (eta:     ?)

In [ ]:
# Surface water  Annual FY 2018 
import ee
image2 = ee.Image('JRC/GSW1_4/YearlyHistory/2018')   #.select(['Daytime', 'Nighttime'])

#geemap.download_ee_image(image2, "SurfaceWater_2018_30_30.tif", region = worldGeometry, scale=30)

j=4
for i in range(6):
  worldGeometry = g[i]  #g[i]
  p1=  str(i)      #str(gstart[i])+"_"+str(gend[i]) # "0_n180"
  filenm =  fl[j]+"_"+p1+".tif"
  geemap.download_ee_image(image2,  filenm, region = worldGeometry, scale=scl)

SurfaceWater_2018_30m_30_0.tif: |          | 0.00/1.42G (raw) [  0.0%] in 00:00 (eta:     ?)

SurfaceWater_2018_30m_30_1.tif: |          | 0.00/1.42G (raw) [  0.0%] in 00:00 (eta:     ?)

SurfaceWater_2018_30m_30_2.tif: |          | 0.00/1.42G (raw) [  0.0%] in 00:00 (eta:     ?)

SurfaceWater_2018_30m_30_3.tif: |          | 0.00/1.42G (raw) [  0.0%] in 00:00 (eta:     ?)

SurfaceWater_2018_30m_30_4.tif: |          | 0.00/1.42G (raw) [  0.0%] in 00:00 (eta:     ?)

SurfaceWater_2018_30m_30_5.tif: |          | 0.00/1.42G (raw) [  0.0%] in 00:00 (eta:     ?)

In [ ]:
# Solar radiation daily May 01 2018  # not used
worldGeometry = ee.Geometry.Polygon([
  [-180, 90],  #// Top-left corner
  [180, 90],   #// Top-right corner
  [180, -90],  #// Bottom-right corner
  [-180, -90]  #// Bottom-left corner
])
cty = ee.FeatureCollection('projects/ee-pradnyauoa/assets/data')

out_dir = os.path.expanduser("~/Downloads")
loc = worldGeometry
collection = ee.ImageCollection('MODIS/061/MCD18A1') \
                  .filter(ee.Filter.date('2018-05-01', '2018-05-02'))
geemap.download_ee_image_collection(collection, out_dir, region=worldGeometry, scale=3000)

Total number of images: 1



2018_05_01.tif: |          | 0.00/4.27G (raw) [  0.0%] in 00:00 (eta:     ?)

In [ ]:
# NDVI Annual FY 2018  # not used
worldGeometry = ee.Geometry.Polygon([
  [-180, 90],  #// Top-left corner
  [180, 90],   #// Top-right corner
  [180, -90],  #// Bottom-right corner
  [-180, -90]  #// Bottom-left corner
])
cty = ee.FeatureCollection('projects/ee-pradnyauoa/assets/data')

out_dir = os.path.expanduser("~/Downloads")
loc = worldGeometry

collection = ee.ImageCollection('LANDSAT/COMPOSITES/C02/T1_L2_ANNUAL_NDVI').filterDate('2018-01-01', '2018-12-31').select("NDVI");

geemap.download_ee_image_collection(collection, out_dir, region=worldGeometry, scale=30, crs='EPSG:4326')


Total number of images: 1



2018.tif: |          | 0.00/5.34M (raw) [  0.0%] in 00:00 (eta:     ?)

In [ ]:
pip install rasterio geopandas

In [74]:
import rasterio
from rasterio import features
from shapely.geometry import box #, mapping
#import geojson
import pandas as pd
import time
import os
start_time = time.time()
print(start_time)



#data.zip to be imported in Google earth engine as asset and then referenced below
cty = ee.FeatureCollection('projects/ee-pradnyauoa/assets/data')
# Filter cities that intersect with the image
#intersecting_cities = cty.filterBounds(image2.geometry())
rsterfiles=["Global_NDVI-0000000000-0000000000.tif", "Global_NDVI-0000000000-0000023296.tif", "Global_NDVI-0000000000-0000046592.tif",
            "Global_NDVI-0000000000-0000069888.tif", "Global_NDVI-0000000000-0000093184.tif", "Global_NDVI-0000000000-0000116480.tif",
            "Global_NDVI-0000000000-0000139776.tif", "Global_NDVI-0000023296-0000000000.tif", "Global_NDVI-0000023296-0000023296.tif",
            "Global_NDVI-0000023296-0000046592.tif", "Global_NDVI-0000023296-0000069888.tif", "Global_NDVI-0000023296-0000093184.tif",
            "Global_NDVI-0000023296-0000116480.tif", "Global_NDVI-0000023296-0000139776.tif", "Global_NDVI-0000046592-0000000000.tif",
            "Global_NDVI-0000046592-0000023296.tif", "Global_NDVI-0000046592-0000046592.tif", "Global_NDVI-0000046592-0000069888.tif",
            "Global_NDVI-0000046592-0000093184.tif", "Global_NDVI-0000046592-0000116480.tif", "Global_NDVI-0000046592-0000139776.tif",
            ]   #".tif", ".tif", ".tif",  #"Global_NDVI-0000046592-0000023296.tif", removed due to error

rsterfiles=["Global_NDVI-0000046592-0000046592.tif", "Global_NDVI-0000046592-0000069888.tif",
            "Global_NDVI-0000046592-0000093184.tif", "Global_NDVI-0000046592-0000116480.tif", "Global_NDVI-0000046592-0000139776.tif",
            ] 

i=-1
i_start =1
if os.path.exists('processed_NDVI_files.csv.csv'):
    cnt= pd.read_csv('processed_NDVI_files.csv.csv', index=False, encoding="utf-8")
    i_start = cnt[0][0]+1
    df= pd.read_csv('City_NDVI_files.csv', index=False, encoding="utf-8")
    print(i_start)
else:
    citylist= list()

i= 15
i_start =15

for fl in rsterfiles:
    if i < i_start:
        i+=1
        print('i=', i)
        continue       
        #  next

    i+=1
    #if i >
    # Open the GeoTIFF file
    print(fl)
    pth = '../data/10_raw_Other_variables/NDVI/'
    

    with rasterio.open(pth+fl) as src:
        # Get the bounding box
        bounds = src.bounds

        coords = [[bounds.left, bounds.bottom],
                  [bounds.left, bounds.top],
                  [bounds.right, bounds.top],
                  [bounds.right, bounds.bottom],
                  [bounds.left, bounds.bottom]]
    
        # Create a valid Earth Engine geometry
        geom_ee = ee.Geometry.Polygon(coords)
    
        tbl = cty.filterBounds(geom_ee)
        ftrs = tbl.toList(tbl.size())
        #print(ftrs))
        fld = 'name_conve'
        # Print the specific property of each feature
        ctylst=list()
        for i in range(0, ftrs.size().getInfo()):
          #print(i)
          feature = ee.Feature(ftrs.get(i))
          #print(feature.geometry)
          ctynm = feature.get(fld).getInfo()
          #ctylst.append(ctynm)
          #print(i, ctynm)
          ctylst.append(ctynm)
          #break
        print(len(ctylst))
        citylist.append(ctylst )
        merged_ndvi_df =  pd.DataFrame(citylist)  #.sort_values(by='0')
        merged_ndvi_df.to_csv('City_NDVI_files.csv', index=False, encoding="utf-8")
        filesdf = pd.DataFrame([len(citylist)])
        filesdf.to_csv('processed_NDVI_files.csv', index=False, encoding="utf-8")
        
print(citylist)

end_time =  time.time()
print("duration", end_time- start_time)
#df = pd.dataframe(citylist)
k= 1
dflist=[]
for ls in citylist:
  lsdf = pd.DataFrame(ls)
  lsdf["NDVIfile"] = k
  k+=1
  dflist.append(lsdf)


#merged_df =  pd.concat(dflist, ignore_index=True).sort_values(by='city')
#merged_df.to_csv('City_NDVI_files.csv', index=False)


1732552347.8486927
Global_NDVI-0000046592-0000046592.tif
456
Global_NDVI-0000046592-0000069888.tif
90
Global_NDVI-0000046592-0000093184.tif
66
Global_NDVI-0000046592-0000116480.tif
34
Global_NDVI-0000046592-0000139776.tif
185
[['Chonchi', 'Comondante Luis Piedrabuena', 'Punta del Este', 'Cuevo', 'Baltasar Brum', 'Paso Rio Mayo', 'Arroyos y Esteros', 'Tinogasta', 'Trancas', 'Calbuco', 'Betanzos', 'Comallo', 'Combarbala', 'Tarabuco', 'Villa Rumipal', 'Chumbicha', 'Maria Elena', 'Uncia', 'Santa Barbara3', 'Villarica', 'Aigua', 'La Paloma', 'Tranqueras', 'Uspallata', 'General Conesa', 'Vergara', 'General Roca', 'Collipulli', 'San Lorenzo1', 'Abra Pampa', 'Caazapa', 'Pampa del Infierno', 'Paraguari', 'San Pedro2', 'Ypacarai|Ita', 'Nueva Imperial', 'Quirihue', 'El Maiten', 'Loncoche', 'Camiri', 'Uyuni', 'Chepes', 'San Juan Bautista', 'Villa Hayes', 'Comandante Fontana', 'Chuquicamata', 'Coronel Bogado', 'Santa Lucia', 'Pedro Luro', 'Artigas', 'Paso de los Toros', 'Rio Bueno', 'La Ligua', 'Ch

In [60]:
def updcsv():
    k= 1
    dflist=[]
    for ls in citylist:
      lsdf = pd.DataFrame(ls)
      lsdf["NDVIfile"] = k
      k+=1
      dflist.append(lsdf)

    merged_ndvi_df =  pd.concat(dflist, ignore_index=True )  #.sort_values(by='0')
    merged_ndvi_df.to_csv('City_NDVI_files.csv', index=False, encoding="utf-8")
    filesdf = pd.DataFrame([len(citylist)])
    filesdf.to_csv('processed_NDVI_files.csv', index=False, encoding="utf-8")
    
print('upd')
t= updcsv()
#merged_df =  pd.concat(dflist, ignore_index=True).sort_values(by='city')
#merged_df.to_csv('City_NDVI_files.csv', index=False)

upd


In [62]:
print(citylist[0])

['Eagle', 'Emmonak', 'Lavrentiya', 'Mekoryuk', 'Nenana', 'Teller', 'Sand Point', 'Selawik', 'Tununak', 'Dillingham', 'Palmer', 'Terrace', 'Kaktovik', 'Chignik', 'Hooper Bay', 'Karluk', 'McGrath', 'Togiak', 'Wales', 'Yakutat', 'Quinhagak', 'Shishmaref', 'Wainwright', 'Hoonah', 'King Salmon', 'Koyuk', 'Unalakleet', 'Unalaska', 'Kotlit', 'Skagway', 'Provideniya', 'Wrangell', 'Cordova', 'Fort Yukon', 'Kotzebue', 'Homer', 'Kenai', 'Ketchikan', 'Point Hope', 'Sitka', 'Fort Good Hope', 'Valdez', 'Nome', 'Barrow', 'Bethel', 'Kodiak', 'Whitehorse', 'Juneau', 'Fairbanks', 'Anchorage']


In [ ]:
print(citylist[1])

In [ ]:
print(len(citylist[3]))


merged_ndvi_df =  pd.concat((citylist ignore_index=True )  #.sort_values(by='0')
merged_ndvi_df.to_csv('City_NDVI_files.csv', index=False, encoding="utf-8")


611


In [ ]:
print(dflist)

[                 0  NDVIfile
0            Eagle         1
1          Emmonak         1
2       Lavrentiya         1
3         Mekoryuk         1
4           Nenana         1
5           Teller         1
6       Sand Point         1
7          Selawik         1
8          Tununak         1
9       Dillingham         1
10          Palmer         1
11         Terrace         1
12        Kaktovik         1
13         Chignik         1
14      Hooper Bay         1
15          Karluk         1
16         McGrath         1
17          Togiak         1
18           Wales         1
19         Yakutat         1
20       Quinhagak         1
21      Shishmaref         1
22      Wainwright         1
23          Hoonah         1
24     King Salmon         1
25           Koyuk         1
26      Unalakleet         1
27        Unalaska         1
28          Kotlit         1
29         Skagway         1
30     Provideniya         1
31        Wrangell         1
32         Cordova         1
33      Fort 

In [ ]:
pip install geojson

In [14]:
import pandas as pd

# Reading a CSV file
df = pd.read_csv('City_NDVI_files.csv', encoding="utf-8")

rsterfiles=["Global_NDVI-0000000000-0000000000.tif", "Global_NDVI-0000000000-0000023296.tif", "Global_NDVI-0000000000-0000046592.tif",
            "Global_NDVI-0000000000-0000069888.tif", "Global_NDVI-0000000000-0000093184.tif", "Global_NDVI-0000000000-0000116480.tif",
            "Global_NDVI-0000000000-0000139776.tif", "Global_NDVI-0000023296-0000000000.tif", "Global_NDVI-0000023296-0000023296.tif",
            "Global_NDVI-0000023296-0000046592.tif", "Global_NDVI-0000023296-0000069888.tif", "Global_NDVI-0000023296-0000093184.tif",
            "Global_NDVI-0000023296-0000116480.tif", "Global_NDVI-0000023296-0000139776.tif", "Global_NDVI-0000046592-0000000000.tif",
            "Global_NDVI-0000046592-0000023296.tif", "Global_NDVI-0000046592-0000046592.tif", "Global_NDVI-0000046592-0000069888.tif",
            "Global_NDVI-0000046592-0000093184.tif", "Global_NDVI-0000046592-0000116480.tif", "Global_NDVI-0000046592-0000139776.tif",
            ]
print((df["NDVIfile"])-1)
x= df["NDVIfile"]-1
print(  rsterfiles[ x[0] ])

df["ndvi_file_name"] = df.apply(lambda row: rsterfiles[pd.to_numeric(row.NDVIfile)-1], axis=1)   #rsterfiles[ pd.to_numeric(df["NDVIfile"])-1]

#(df)

#for index, rows in df.iterrows():
print(df)
df.to_csv('City_NDVI_files.csv')



0        0
1        0
2        0
3        0
4        0
        ..
6072    20
6073    20
6074    20
6075    20
6076    20
Name: NDVIfile, Length: 6077, dtype: int64
Global_NDVI-0000000000-0000000000.tif
               0  NDVIfile                         ndvi_file_name
0          Eagle         1  Global_NDVI-0000000000-0000000000.tif
1        Emmonak         1  Global_NDVI-0000000000-0000000000.tif
2     Lavrentiya         1  Global_NDVI-0000000000-0000000000.tif
3       Mekoryuk         1  Global_NDVI-0000000000-0000000000.tif
4         Nenana         1  Global_NDVI-0000000000-0000000000.tif
...          ...       ...                                    ...
6072    Canberra        21  Global_NDVI-0000046592-0000139776.tif
6073    Adelaide        21  Global_NDVI-0000046592-0000139776.tif
6074    Brisbane        21  Global_NDVI-0000046592-0000139776.tif
6075  Melbourne2        21  Global_NDVI-0000046592-0000139776.tif
6076     Sydney1        21  Global_NDVI-0000046592-0000139776.tif

[6077